In [1]:
import vectorbt as vbt
from vectorbt.portfolio.enums import (
    SizeType, 
    ConflictMode, 
    CallSeqType, 
    Order, 
    NoOrder, 
    InitCashMode,
    SignalType
)
from vectorbt.records.enums import OrderSide

import numpy as np
import pandas as pd
from numba import njit
from datetime import datetime, timedelta

seed = 42

price = pd.Series([1., 2., 3., 4., 5.], index=pd.Index([
    datetime(2020, 1, 1),
    datetime(2020, 1, 2),
    datetime(2020, 1, 3),
    datetime(2020, 1, 4),
    datetime(2020, 1, 5)
]))
price_wide = price.vbt.tile(3, keys=['a', 'b', 'c'])
print(price.shape, price_wide.shape)

big_price = pd.DataFrame(np.random.uniform(size=(1000,)))
big_price.index = [datetime(2018, 1, 1) + timedelta(days=i) for i in range(1000)]
big_price_wide = big_price.vbt.tile(1000)
print(big_price.shape, big_price_wide.shape)

# Disable caching for performance testing
# NOTE: Expect waterfall of executions, since some attributes depend on other attributes 
# that have to be calculated again and again
vbt.defaults.caching = False

(5,) (5, 3)
(1000, 1) (1000, 1000)


## from_signals

In [2]:
entries = pd.Series([True, True, True, False, False], index=price.index)
entries_wide = entries.vbt.tile(3, keys=['a', 'b', 'c'])
print(entries.shape, entries_wide.shape)

exits = pd.Series([False, False, True, True, True], index=price.index)
exits_wide = exits.vbt.tile(3, keys=['a', 'b', 'c'])
print(exits.shape, exits_wide.shape)

big_entries = pd.DataFrame.vbt.signals.empty((1000,), index=big_price.index)
big_entries.iloc[0::2] = True
big_entries_wide = big_entries.vbt.tile(1000)
print(big_entries.shape, big_entries_wide.shape)

big_exits = pd.DataFrame.vbt.signals.empty((1000,), index=big_price.index)
big_exits.iloc[1::2] = True
big_exits_wide = big_exits.vbt.tile(1000)
print(big_exits.shape, big_exits_wide.shape)

(5,) (5, 3)
(5,) (5, 3)
(1000,) (1000, 1000)
(1000,) (1000, 1000)


In [3]:
%timeit vbt.Portfolio.from_signals(\
    big_price, big_entries, big_exits,\
    size=1., init_cash=np.inf, accumulate=True)

2.37 ms ± 523 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%timeit vbt.Portfolio.from_signals(\
    big_price_wide, big_entries_wide, big_exits_wide,\
    size=1., init_cash=np.inf, accumulate=True)

97.3 ms ± 6.46 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%timeit vbt.Portfolio.from_signals(\
    big_price_wide, big_entries_wide, big_exits_wide,\
    size=1., init_cash=np.inf, accumulate=True,\
    group_by=np.repeat(np.arange(500), 2))

97.8 ms ± 5.83 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
%timeit vbt.Portfolio.from_signals(\
    big_price_wide, big_entries_wide, big_exits_wide,\
    size=1., init_cash=np.inf, accumulate=True,\
    group_by=np.repeat(np.arange(500), 2), cash_sharing=True)

128 ms ± 23.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
# worst case
%timeit vbt.Portfolio.from_signals(\
    big_price_wide, big_entries_wide, big_exits_wide,\
    size=np.arange(1000)[::-1], init_cash=np.inf, accumulate=True,\
    group_by=np.repeat(np.arange(500), 2), cash_sharing=True, call_seq=CallSeqType.Auto)

123 ms ± 9.05 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit vbt.Portfolio.from_signals(\
    big_price_wide, big_entries_wide, big_exits_wide,\
    size=1., init_cash=np.inf, accumulate=True,\
    group_by=np.full(1000, 0))

111 ms ± 17.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%timeit vbt.Portfolio.from_signals(\
    big_price_wide, big_entries_wide, big_exits_wide,\
    size=1., init_cash=np.inf, accumulate=True,\
    group_by=np.full(1000, 0), cash_sharing=True)

108 ms ± 8.17 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
# worst case
%timeit vbt.Portfolio.from_signals(\
    big_price_wide, big_entries_wide, big_exits_wide,\
    size=np.arange(1000)[::-1], init_cash=np.inf, accumulate=True,\
    group_by=np.full(1000, 0), cash_sharing=True, call_seq=CallSeqType.Auto)

643 ms ± 55.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
portfolio = vbt.Portfolio.from_signals(price, entries, exits)

print(portfolio.orders().records)

   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    0    3  100.0    4.0   0.0     1


In [4]:
portfolio = vbt.Portfolio.from_signals(price_wide, entries, exits)

print(portfolio.orders().records)

   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    0    3  100.0    4.0   0.0     1
2    1    0  100.0    1.0   0.0     0
3    1    3  100.0    4.0   0.0     1
4    2    0  100.0    1.0   0.0     0
5    2    3  100.0    4.0   0.0     1


In [5]:
def test_per_signal_type(price=price, entries=entries, exits=exits, val_price=price, **kwargs):
    print('Long')
    try:
        portfolio = vbt.Portfolio.from_signals(price, entries, exits, signal_type='long', **kwargs)
        print(portfolio.orders().records)
    except Exception as e:
        print(e)
    print('Short')
    try:
        portfolio = vbt.Portfolio.from_signals(price, entries, exits, signal_type='short', **kwargs)
        print(portfolio.orders().records)
    except Exception as e:
        print(e)
    print('LongShort')
    try:
        portfolio = vbt.Portfolio.from_signals(price, entries, exits, signal_type='longshort', **kwargs)
        print(portfolio.orders().records)
    except Exception as e:
        print(e)

In [6]:
test_per_signal_type(size=[[-1, 0, np.inf]])
test_per_signal_type(long_size=1., short_size=2.)

Long
   col  idx   size  price  fees  side
0    0    0    1.0    1.0   0.0     0
1    0    3    1.0    4.0   0.0     1
2    2    0  100.0    1.0   0.0     0
3    2    3  100.0    4.0   0.0     1
Short
   col  idx   size  price  fees  side
0    0    0    1.0    1.0   0.0     1
1    0    3    1.0    4.0   0.0     0
2    2    0  100.0    1.0   0.0     1
3    2    3   50.0    4.0   0.0     0
LongShort
   col  idx   size  price  fees  side
0    0    0    1.0    1.0   0.0     0
1    0    3    2.0    4.0   0.0     1
2    2    0  100.0    1.0   0.0     0
3    2    3  200.0    4.0   0.0     1
Long
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    3   1.0    4.0   0.0     1
Short
   col  idx  size  price  fees  side
0    0    0   2.0    1.0   0.0     1
1    0    3   2.0    4.0   0.0     0
LongShort
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    3   3.0    4.0   0.0     1


In [7]:
test_per_signal_type(size=1, long_price=price*1.01, short_price=price*0.99)

Long
   col  idx  size  price  fees  side
0    0    0   1.0   1.01   0.0     0
1    0    3   1.0   3.96   0.0     1
Short
   col  idx  size  price  fees  side
0    0    0   1.0   0.99   0.0     1
1    0    3   1.0   4.04   0.0     0
LongShort
   col  idx  size  price  fees  side
0    0    0   1.0   1.01   0.0     0
1    0    3   2.0   3.96   0.0     1


In [8]:
test_per_signal_type(size=1, fees=[[0., 0.1, 1.]])
test_per_signal_type(size=1, long_fees=0., short_fees=1.)

Long
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    3   1.0    4.0   0.0     1
2    1    0   1.0    1.0   0.1     0
3    1    3   1.0    4.0   0.4     1
4    2    0   1.0    1.0   1.0     0
5    2    3   1.0    4.0   4.0     1
Short
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     1
1    0    3   1.0    4.0   0.0     0
2    1    0   1.0    1.0   0.1     1
3    1    3   1.0    4.0   0.4     0
4    2    0   1.0    1.0   1.0     1
5    2    3   1.0    4.0   4.0     0
LongShort
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    3   2.0    4.0   0.0     1
2    1    0   1.0    1.0   0.1     0
3    1    3   2.0    4.0   0.8     1
4    2    0   1.0    1.0   1.0     0
5    2    3   2.0    4.0   8.0     1
Long
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    3   1.0    4.0   4.0     1
Short
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   1.0     1
1    0

In [9]:
test_per_signal_type(size=1, fixed_fees=[[0., 0.1, 1.]])
test_per_signal_type(size=1, long_fixed_fees=0., short_fixed_fees=1.)

Long
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    3   1.0    4.0   0.0     1
2    1    0   1.0    1.0   0.1     0
3    1    3   1.0    4.0   0.1     1
4    2    0   1.0    1.0   1.0     0
5    2    3   1.0    4.0   1.0     1
Short
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     1
1    0    3   1.0    4.0   0.0     0
2    1    0   1.0    1.0   0.1     1
3    1    3   1.0    4.0   0.1     0
4    2    0   1.0    1.0   1.0     1
5    2    3   1.0    4.0   1.0     0
LongShort
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    3   2.0    4.0   0.0     1
2    1    0   1.0    1.0   0.1     0
3    1    3   2.0    4.0   0.1     1
4    2    0   1.0    1.0   1.0     0
5    2    3   2.0    4.0   1.0     1
Long
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    3   1.0    4.0   1.0     1
Short
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   1.0     1
1    0

In [13]:
test_per_signal_type(size=1, slippage=[[0., 0.1, 1.]])
test_per_signal_type(size=1, long_slippage=0., short_slippage=0.5)

Long
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    3   1.0    4.0   0.0     1
2    1    0   1.0    1.1   0.0     0
3    1    3   1.0    3.6   0.0     1
4    2    0   1.0    2.0   0.0     0
5    2    3   1.0    0.0   0.0     1
Short
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     1
1    0    3   1.0    4.0   0.0     0
2    1    0   1.0    0.9   0.0     1
3    1    3   1.0    4.4   0.0     0
4    2    0   1.0    0.0   0.0     1
5    2    3   1.0    8.0   0.0     0
LongShort
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    3   2.0    4.0   0.0     1
2    1    0   1.0    1.1   0.0     0
3    1    3   2.0    3.6   0.0     1
4    2    0   1.0    2.0   0.0     0
5    2    3   2.0    0.0   0.0     1
Long
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    3   1.0    2.0   0.0     1
Short
   col  idx  size  price  fees  side
0    0    0   1.0    0.5   0.0     1
1    0

In [14]:
test_per_signal_type(size=1, min_size=[[0., 1., 2.]])
test_per_signal_type(size=1, long_min_size=1., short_min_size=2.)

Long
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    3   1.0    4.0   0.0     1
2    1    0   1.0    1.0   0.0     0
3    1    3   1.0    4.0   0.0     1
Short
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     1
1    0    3   1.0    4.0   0.0     0
2    1    0   1.0    1.0   0.0     1
3    1    3   1.0    4.0   0.0     0
LongShort
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    3   2.0    4.0   0.0     1
2    1    0   1.0    1.0   0.0     0
3    1    3   2.0    4.0   0.0     1
Long
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
Short
Empty DataFrame
Columns: [col, idx, size, price, fees, side]
Index: []
LongShort
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    3   2.0    4.0   0.0     1


In [15]:
test_per_signal_type(size=1, max_size=[[0.5, 1., np.inf]])
test_per_signal_type(size=1, long_max_size=0.5, short_max_size=np.inf)

Long
   col  idx  size  price  fees  side
0    0    0   0.5    1.0   0.0     0
1    0    3   0.5    4.0   0.0     1
2    1    0   1.0    1.0   0.0     0
3    1    3   1.0    4.0   0.0     1
4    2    0   1.0    1.0   0.0     0
5    2    3   1.0    4.0   0.0     1
Short
   col  idx  size  price  fees  side
0    0    0   0.5    1.0   0.0     1
1    0    3   0.5    4.0   0.0     0
2    1    0   1.0    1.0   0.0     1
3    1    3   1.0    4.0   0.0     0
4    2    0   1.0    1.0   0.0     1
5    2    3   1.0    4.0   0.0     0
LongShort
   col  idx  size  price  fees  side
0    0    0   0.5    1.0   0.0     0
1    0    3   0.5    4.0   0.0     1
2    0    4   0.5    5.0   0.0     1
3    1    0   1.0    1.0   0.0     0
4    1    3   1.0    4.0   0.0     1
5    1    4   1.0    5.0   0.0     1
6    2    0   1.0    1.0   0.0     0
7    2    3   2.0    4.0   0.0     1
Long
   col  idx  size  price  fees  side
0    0    0   0.5    1.0   0.0     0
1    0    3   0.5    4.0   0.0     1
Short
   col

In [16]:
test_per_signal_type(size=1., reject_prob=[[0., 0.5, 1.]], seed=42)
test_per_signal_type(size=1, long_reject_prob=0., short_reject_prob=1., seed=42)

Long
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    3   1.0    4.0   0.0     1
2    1    1   1.0    2.0   0.0     0
3    1    3   1.0    4.0   0.0     1
Short
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     1
1    0    3   1.0    4.0   0.0     0
2    1    1   1.0    2.0   0.0     1
3    1    3   1.0    4.0   0.0     0
LongShort
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    3   2.0    4.0   0.0     1
2    1    1   1.0    2.0   0.0     0
3    1    3   2.0    4.0   0.0     1
Long
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
Short
Empty DataFrame
Columns: [col, idx, size, price, fees, side]
Index: []
LongShort
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0


In [8]:
test_per_signal_type(size=1000, allow_partial=[[True, False]])
test_per_signal_type(size=1000, long_allow_partial=False, short_allow_partial=True)
test_per_signal_type(size=np.inf, long_allow_partial=False, short_allow_partial=True)

Long
   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    0    3  100.0    4.0   0.0     1
Short
   col  idx    size  price  fees  side
0    0    0  1000.0    1.0   0.0     1
1    0    3   275.0    4.0   0.0     0
2    1    0  1000.0    1.0   0.0     1
LongShort
   col  idx    size  price  fees  side
0    0    0   100.0    1.0   0.0     0
1    0    3  1100.0    4.0   0.0     1
2    1    3  1000.0    4.0   0.0     1
Long
Empty DataFrame
Columns: [col, idx, size, price, fees, side]
Index: []
Short
   col  idx    size  price  fees  side
0    0    0  1000.0    1.0   0.0     1
LongShort
   col  idx    size  price  fees  side
0    0    3  1000.0    4.0   0.0     1
Long
   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    0    3  100.0    4.0   0.0     1
Short
   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     1
LongShort
   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    0    3  200.0

In [9]:
test_per_signal_type(size=1000, raise_by_reject=True)
test_per_signal_type(size=1000, allow_partial=False, raise_by_reject=True)
test_per_signal_type(size=1000, allow_partial=False, long_raise_by_reject=True, short_raise_by_reject=False)

Long
   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    0    3  100.0    4.0   0.0     1
Short
Order rejected due to insufficient funds
LongShort
   col  idx    size  price  fees  side
0    0    0   100.0    1.0   0.0     0
1    0    3  1100.0    4.0   0.0     1
Long
Order rejected due to partial fill
Short
Order rejected due to partial fill
LongShort
Order rejected due to partial fill
Long
Order rejected due to partial fill
Short
Order rejected due to partial fill
LongShort
Order rejected due to partial fill


In [10]:
portfolio = vbt.Portfolio.from_signals(
    price, entries, exits, signal_type=SignalType.LongShort,
    close_first=[[False, True]]
)
print(portfolio.orders().records)

portfolio = vbt.Portfolio.from_signals(
    pd.Series(price.values[::-1], index=price.index),
    pd.Series(entries.values[::-1], index=price.index),
    pd.Series(exits.values[::-1], index=price.index), 
    signal_type=SignalType.LongShort,
    close_first=[[False, True]]
)
print(portfolio.orders().records)

   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    0    3  200.0    4.0   0.0     1
2    1    0  100.0    1.0   0.0     0
3    1    3  100.0    4.0   0.0     1
4    1    4   80.0    5.0   0.0     1
   col  idx   size  price  fees  side
0    0    0   20.0    5.0   0.0     1
1    0    3  100.0    2.0   0.0     0
2    1    0   20.0    5.0   0.0     1
3    1    3   20.0    2.0   0.0     0
4    1    4  160.0    1.0   0.0     0


In [21]:
test_per_signal_type(size=1., accumulate=[[False, True]])
test_per_signal_type(size=1, long_accumulate=True, short_accumulate=False)

Long
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    3   1.0    4.0   0.0     1
2    1    0   1.0    1.0   0.0     0
3    1    1   1.0    2.0   0.0     0
4    1    3   1.0    4.0   0.0     1
5    1    4   1.0    5.0   0.0     1
Short
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     1
1    0    3   1.0    4.0   0.0     0
2    1    0   1.0    1.0   0.0     1
3    1    1   1.0    2.0   0.0     1
4    1    3   1.0    4.0   0.0     0
5    1    4   1.0    5.0   0.0     0
LongShort
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    3   2.0    4.0   0.0     1
2    1    0   1.0    1.0   0.0     0
3    1    1   1.0    2.0   0.0     0
4    1    3   1.0    4.0   0.0     1
5    1    4   1.0    5.0   0.0     1
Long
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    1   1.0    2.0   0.0     0
2    0    3   2.0    4.0   0.0     1
Short
   col  idx  size  price  fees  side
0    0

In [11]:
test_per_signal_type(price=price.iloc[:3], entries=pd.DataFrame([
    [True, True, True, True, True],
    [True, True, True, True, False],
    [True, True, True, True, True]
]), exits=pd.DataFrame([
    [True, True, True, True, True],
    [False, False, False, False, True],
    [True, True, True, True, True]
]), size=1., conflict_mode=[[
    ConflictMode.Ignore, 
    ConflictMode.Entry, 
    ConflictMode.Exit, 
    ConflictMode.Opposite,
    ConflictMode.Opposite
]])

Long
   col  idx  size  price  fees  side
0    0    1   1.0    2.0   0.0     0
1    1    0   1.0    1.0   0.0     0
2    2    1   1.0    2.0   0.0     0
3    2    2   1.0    3.0   0.0     1
4    3    1   1.0    2.0   0.0     0
5    3    2   1.0    3.0   0.0     1
Short
   col  idx  size  price  fees  side
0    0    1   1.0    2.0   0.0     1
1    1    0   1.0    1.0   0.0     1
2    2    1   1.0    2.0   0.0     1
3    2    2   1.0    3.0   0.0     0
4    3    1   1.0    2.0   0.0     1
5    3    2   1.0    3.0   0.0     0
LongShort
   col  idx  size  price  fees  side
0    0    1   1.0    2.0   0.0     0
1    1    0   1.0    1.0   0.0     0
2    2    0   1.0    1.0   0.0     1
3    2    1   2.0    2.0   0.0     0
4    2    2   2.0    3.0   0.0     1
5    3    1   1.0    2.0   0.0     0
6    3    2   2.0    3.0   0.0     1
7    4    1   1.0    2.0   0.0     1
8    4    2   2.0    3.0   0.0     0


In [11]:
test_per_signal_type(init_cash=0.)
test_per_signal_type(init_cash=np.inf)

Long
Empty DataFrame
Columns: [col, idx, size, price, fees, side]
Index: []
Short
Empty DataFrame
Columns: [col, idx, size, price, fees, side]
Index: []
LongShort
Empty DataFrame
Columns: [col, idx, size, price, fees, side]
Index: []
Long
Attempt to go in long direction indefinitely. Set max_size or finite init_cash.
Short
value_now must be finite
LongShort
Attempt to go in long direction indefinitely. Set max_size or finite init_cash.


In [12]:
portfolio = vbt.Portfolio.from_signals(
    price_wide, entries, exits, 
    group_by=np.array([0, 0, 1]))

print(portfolio.orders().records)

   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    0    3  100.0    4.0   0.0     1
2    1    0  100.0    1.0   0.0     0
3    1    3  100.0    4.0   0.0     1
4    2    0  100.0    1.0   0.0     0
5    2    3  100.0    4.0   0.0     1


In [13]:
portfolio = vbt.Portfolio.from_signals(
    price_wide, entries, exits, 
    group_by=np.array([0, 0, 1]), cash_sharing=True)

print(portfolio.orders().records)
print(portfolio.call_seq)

   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    0    3  100.0    4.0   0.0     1
2    2    0  100.0    1.0   0.0     0
3    2    3  100.0    4.0   0.0     1
            a  b  c
2020-01-01  0  1  0
2020-01-02  0  1  0
2020-01-03  0  1  0
2020-01-04  0  1  0
2020-01-05  0  1  0


In [15]:
portfolio = vbt.Portfolio.from_signals(
    price_wide, entries, exits, 
    group_by=np.array([0, 0, 1]), cash_sharing=True, call_seq=CallSeqType.Reversed)

print(portfolio.orders().records)
print(portfolio.call_seq)

   col  idx   size  price  fees  side
0    1    0  100.0    1.0   0.0     0
1    1    3  100.0    4.0   0.0     1
2    2    0  100.0    1.0   0.0     0
3    2    3  100.0    4.0   0.0     1
            a  b  c
2020-01-01  1  0  0
2020-01-02  1  0  0
2020-01-03  1  0  0
2020-01-04  1  0  0
2020-01-05  1  0  0


In [16]:
portfolio = vbt.Portfolio.from_signals(
    price_wide, entries, exits, 
    group_by=np.array([0, 0, 1]), cash_sharing=True, call_seq=CallSeqType.Random)

print(portfolio.orders().records)
print(portfolio.call_seq)

portfolio = vbt.Portfolio.from_signals(
    price_wide, entries, exits, 
    group_by=np.array([0, 0, 1]), cash_sharing=True, call_seq=CallSeqType.Random, seed=42)

print(portfolio.orders().records)
print(portfolio.call_seq)

   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    0    3  100.0    4.0   0.0     1
2    2    0  100.0    1.0   0.0     0
3    2    3  100.0    4.0   0.0     1
            a  b  c
2020-01-01  0  1  0
2020-01-02  1  0  0
2020-01-03  0  1  0
2020-01-04  1  0  0
2020-01-05  0  1  0
   col  idx   size  price  fees  side
0    1    0  100.0    1.0   0.0     0
1    1    3  100.0    4.0   0.0     1
2    2    0  100.0    1.0   0.0     0
3    2    3  100.0    4.0   0.0     1
            a  b  c
2020-01-01  1  0  0
2020-01-02  0  1  0
2020-01-03  1  0  0
2020-01-04  1  0  0
2020-01-05  1  0  0


In [14]:
for signal_type in ('long', 'short', 'longshort'):
    portfolio = vbt.Portfolio.from_signals(price_wide, pd.DataFrame([
        [False, False, True],
        [False, True, False],
        [True, False, False],
        [False, False, True],
        [False, True, False],
    ]), pd.DataFrame([
        [False, False, False],
        [False, False, True],
        [False, True, False],
        [True, False, False],
        [False, False, True],
    ]), group_by=np.array([0, 0, 0]), 
        cash_sharing=True, call_seq=CallSeqType.Auto, signal_type=signal_type)

    print(portfolio.orders().records)
    print(portfolio.call_seq)
    print(portfolio.shares())

   col  idx   size  price  fees  side
0    0    2  100.0    3.0   0.0     0
1    0    3  100.0    4.0   0.0     1
2    1    1  100.0    2.0   0.0     0
3    1    2  100.0    3.0   0.0     1
4    1    4  100.0    5.0   0.0     0
5    2    0  100.0    1.0   0.0     0
6    2    1  100.0    2.0   0.0     1
7    2    3  100.0    4.0   0.0     0
8    2    4  100.0    5.0   0.0     1
            a  b  c
2020-01-01  0  1  2
2020-01-02  2  0  1
2020-01-03  1  2  0
2020-01-04  0  1  2
2020-01-05  2  0  1
                a      b      c
2020-01-01    0.0    0.0  100.0
2020-01-02    0.0  100.0    0.0
2020-01-03  100.0    0.0    0.0
2020-01-04    0.0    0.0  100.0
2020-01-05    0.0  100.0    0.0
   col  idx   size  price  fees  side
0    2    0  100.0    1.0   0.0     1
1    2    1  100.0    2.0   0.0     0
            a  b  c
2020-01-01  2  0  1
2020-01-02  1  0  2
2020-01-03  0  1  2
2020-01-04  2  0  1
2020-01-05  1  0  2
              a    b      c
2020-01-01  0.0  0.0 -100.0
2020-01-02  0.0  0

## from_orders

In [3]:
order_size = pd.Series([np.inf, -np.inf, np.nan, np.inf, -np.inf], index=price.index)
order_size_wide = order_size.vbt.tile(3, keys=['a', 'b', 'c'])

big_order_size = pd.DataFrame.vbt.empty((1000,), 1, dtype=np.float_)
big_order_size.iloc[1::2] = -1
big_order_size_wide = big_order_size.vbt.tile(1000)

In [8]:
%timeit vbt.Portfolio.from_orders(big_price, big_order_size, init_cash=np.inf)

1.58 ms ± 174 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%timeit vbt.Portfolio.from_orders(big_price_wide, big_order_size_wide, init_cash=np.inf)

83.2 ms ± 8.25 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%timeit vbt.Portfolio.from_orders(\
    big_price_wide, big_order_size_wide, init_cash=np.inf, \
    group_by=np.repeat(np.arange(500), 2))

86.9 ms ± 9.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [31]:
%timeit vbt.Portfolio.from_orders(\
    big_price_wide, big_order_size_wide, init_cash=np.inf, \
    group_by=np.repeat(np.arange(500), 2), cash_sharing=True)

110 ms ± 21.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%timeit vbt.Portfolio.from_orders(\
    big_price_wide, big_order_size_wide, init_cash=np.inf, \
    group_by=np.repeat(np.arange(500), 2), cash_sharing=True, call_seq=CallSeqType.Auto)

115 ms ± 3.86 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
%timeit vbt.Portfolio.from_orders(\
    big_price_wide, big_order_size_wide, init_cash=np.inf, \
    group_by=np.full(1000, 0))

86.2 ms ± 5.46 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
%timeit vbt.Portfolio.from_orders(\
    big_price_wide, big_order_size_wide, init_cash=np.inf, \
    group_by=np.full(1000, 0), cash_sharing=True)

88.7 ms ± 7.49 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
%timeit vbt.Portfolio.from_orders(\
    big_price_wide, big_order_size_wide, init_cash=np.inf, \
    group_by=np.full(1000, 0), cash_sharing=True, call_seq=CallSeqType.Auto)

94 ms ± 3.54 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
portfolio = vbt.Portfolio.from_orders(price, order_size)

print(portfolio.orders().records)

   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    0    1  200.0    2.0   0.0     1
2    0    3  100.0    4.0   0.0     0


In [14]:
portfolio = vbt.Portfolio.from_orders(price_wide, order_size)

print(portfolio.orders().records)

   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    0    1  200.0    2.0   0.0     1
2    0    3  100.0    4.0   0.0     0
3    1    0  100.0    1.0   0.0     0
4    1    1  200.0    2.0   0.0     1
5    1    3  100.0    4.0   0.0     0
6    2    0  100.0    1.0   0.0     0
7    2    1  200.0    2.0   0.0     1
8    2    3  100.0    4.0   0.0     0


In [15]:
portfolio = vbt.Portfolio.from_orders(
    price_wide, order_size, 
    group_by=np.array([0, 0, 1]))

print(portfolio.orders().records)

   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    0    1  200.0    2.0   0.0     1
2    0    3  100.0    4.0   0.0     0
3    1    0  100.0    1.0   0.0     0
4    1    1  200.0    2.0   0.0     1
5    1    3  100.0    4.0   0.0     0
6    2    0  100.0    1.0   0.0     0
7    2    1  200.0    2.0   0.0     1
8    2    3  100.0    4.0   0.0     0


In [16]:
portfolio = vbt.Portfolio.from_orders(price, -np.inf)
print(portfolio.orders().records)

   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     1


In [17]:
portfolio = vbt.Portfolio.from_orders(
    price_wide, np.inf, 
    group_by=np.array([0, 0, 1]), cash_sharing=True)

print(portfolio.orders().records)

portfolio = vbt.Portfolio.from_orders(
    price_wide, -np.inf, 
    group_by=np.array([0, 0, 1]), cash_sharing=True)

print(portfolio.orders().records)
print(portfolio.cash())

   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    2    0  100.0    1.0   0.0     0
   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     1
1    1    0  100.0    1.0   0.0     1
2    2    0  100.0    1.0   0.0     1
                0      1
2020-01-01  300.0  200.0
2020-01-02  300.0  200.0
2020-01-03  300.0  200.0
2020-01-04  300.0  200.0
2020-01-05  300.0  200.0


In [18]:
portfolio = vbt.Portfolio.from_orders(
    price_wide, order_size, 
    group_by=np.array([0, 0, 1]), cash_sharing=True)

print(portfolio.orders().records)
print(portfolio.call_seq)

   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    0    1  200.0    2.0   0.0     1
2    0    3  150.0    4.0   0.0     0
3    1    1  100.0    2.0   0.0     1
4    2    0  100.0    1.0   0.0     0
5    2    1  200.0    2.0   0.0     1
6    2    3  100.0    4.0   0.0     0
            a  b  c
2020-01-01  0  1  0
2020-01-02  0  1  0
2020-01-03  0  1  0
2020-01-04  0  1  0
2020-01-05  0  1  0


In [19]:
portfolio = vbt.Portfolio.from_orders(
    price_wide, order_size, 
    group_by=np.array([0, 0, 1]), cash_sharing=True, call_seq=CallSeqType.Reversed)

print(portfolio.orders().records)
print(portfolio.call_seq)

   col  idx   size  price  fees  side
0    0    1  100.0    2.0   0.0     1
1    1    0  100.0    1.0   0.0     0
2    1    1  200.0    2.0   0.0     1
3    1    3  150.0    4.0   0.0     0
4    2    0  100.0    1.0   0.0     0
5    2    1  200.0    2.0   0.0     1
6    2    3  100.0    4.0   0.0     0
            a  b  c
2020-01-01  1  0  0
2020-01-02  1  0  0
2020-01-03  1  0  0
2020-01-04  1  0  0
2020-01-05  1  0  0


In [20]:
portfolio = vbt.Portfolio.from_orders(
    price_wide, order_size, 
    group_by=np.array([0, 0, 1]), cash_sharing=True, call_seq=CallSeqType.Random)

print(portfolio.orders().records)
print(portfolio.call_seq)

portfolio = vbt.Portfolio.from_orders(
    price_wide, order_size, 
    group_by=np.array([0, 0, 1]), cash_sharing=True, call_seq=CallSeqType.Random, seed=42)

print(portfolio.orders().records)
print(portfolio.call_seq)

   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    0    1  200.0    2.0   0.0     1
2    0    3  150.0    4.0   0.0     0
3    1    1  100.0    2.0   0.0     1
4    2    0  100.0    1.0   0.0     0
5    2    1  200.0    2.0   0.0     1
6    2    3  100.0    4.0   0.0     0
            a  b  c
2020-01-01  0  1  0
2020-01-02  1  0  0
2020-01-03  0  1  0
2020-01-04  0  1  0
2020-01-05  0  1  0
   col  idx   size  price  fees  side
0    0    1  100.0    2.0   0.0     1
1    1    0  100.0    1.0   0.0     0
2    1    1  200.0    2.0   0.0     1
3    1    3  150.0    4.0   0.0     0
4    2    0  100.0    1.0   0.0     0
5    2    1  200.0    2.0   0.0     1
6    2    3  100.0    4.0   0.0     0
            a  b  c
2020-01-01  1  0  0
2020-01-02  0  1  0
2020-01-03  1  0  0
2020-01-04  1  0  0
2020-01-05  1  0  0


In [14]:
order_size_one = pd.Series([1, -1, np.nan, 1, -1], index=price.index)

In [22]:
portfolio = vbt.Portfolio.from_orders(price_wide, order_size_one, fees=[0., 0.01, 1.])

print(portfolio.orders().records)

    col  idx  size  price  fees  side
0     0    0   1.0    1.0  0.00     0
1     0    1   1.0    2.0  0.00     1
2     0    3   1.0    4.0  0.00     0
3     0    4   1.0    5.0  0.00     1
4     1    0   1.0    1.0  0.01     0
5     1    1   1.0    2.0  0.02     1
6     1    3   1.0    4.0  0.04     0
7     1    4   1.0    5.0  0.05     1
8     2    0   1.0    1.0  1.00     0
9     2    1   1.0    2.0  2.00     1
10    2    3   1.0    4.0  4.00     0
11    2    4   1.0    5.0  5.00     1


In [23]:
portfolio = vbt.Portfolio.from_orders(price_wide, order_size_one, fixed_fees=[0., 0.01, 1.])

print(portfolio.orders().records)

    col  idx  size  price  fees  side
0     0    0   1.0    1.0  0.00     0
1     0    1   1.0    2.0  0.00     1
2     0    3   1.0    4.0  0.00     0
3     0    4   1.0    5.0  0.00     1
4     1    0   1.0    1.0  0.01     0
5     1    1   1.0    2.0  0.01     1
6     1    3   1.0    4.0  0.01     0
7     1    4   1.0    5.0  0.01     1
8     2    0   1.0    1.0  1.00     0
9     2    1   1.0    2.0  1.00     1
10    2    3   1.0    4.0  1.00     0
11    2    4   1.0    5.0  1.00     1


In [24]:
portfolio = vbt.Portfolio.from_orders(price_wide, order_size_one, slippage=[0., 0.01, 1.])

print(portfolio.orders().records)

    col  idx  size  price  fees  side
0     0    0   1.0   1.00   0.0     0
1     0    1   1.0   2.00   0.0     1
2     0    3   1.0   4.00   0.0     0
3     0    4   1.0   5.00   0.0     1
4     1    0   1.0   1.01   0.0     0
5     1    1   1.0   1.98   0.0     1
6     1    3   1.0   4.04   0.0     0
7     1    4   1.0   4.95   0.0     1
8     2    0   1.0   2.00   0.0     0
9     2    1   1.0   0.00   0.0     1
10    2    3   1.0   8.00   0.0     0
11    2    4   1.0   0.00   0.0     1


In [25]:
portfolio = vbt.Portfolio.from_orders(price_wide, order_size_one, min_size=[0., 1., 2.])

print(portfolio.orders().records)

   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    1   1.0    2.0   0.0     1
2    0    3   1.0    4.0   0.0     0
3    0    4   1.0    5.0   0.0     1
4    1    0   1.0    1.0   0.0     0
5    1    1   1.0    2.0   0.0     1
6    1    3   1.0    4.0   0.0     0
7    1    4   1.0    5.0   0.0     1


In [26]:
portfolio = vbt.Portfolio.from_orders(price_wide, order_size_one, max_size=[0.5, 1., np.inf])

print(portfolio.orders().records)

    col  idx  size  price  fees  side
0     0    0   0.5    1.0   0.0     0
1     0    1   0.5    2.0   0.0     1
2     0    3   0.5    4.0   0.0     0
3     0    4   0.5    5.0   0.0     1
4     1    0   1.0    1.0   0.0     0
5     1    1   1.0    2.0   0.0     1
6     1    3   1.0    4.0   0.0     0
7     1    4   1.0    5.0   0.0     1
8     2    0   1.0    1.0   0.0     0
9     2    1   1.0    2.0   0.0     1
10    2    3   1.0    4.0   0.0     0
11    2    4   1.0    5.0   0.0     1


In [27]:
portfolio = vbt.Portfolio.from_orders(price_wide, order_size_one, reject_prob=[0., 0.5, 1.], seed=42)

print(portfolio.orders().records)

   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    1   1.0    2.0   0.0     1
2    0    3   1.0    4.0   0.0     0
3    0    4   1.0    5.0   0.0     1
4    1    1   1.0    2.0   0.0     1
5    1    3   1.0    4.0   0.0     0
6    1    4   1.0    5.0   0.0     1


In [19]:
portfolio = vbt.Portfolio.from_orders(price, 1000, allow_partial=[[True, False]])
print(portfolio.orders().records)

portfolio = vbt.Portfolio.from_orders(price, np.inf, allow_partial=[[True, False]])
print(portfolio.orders().records)

   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    1    0  100.0    1.0   0.0     0


In [22]:
try:
    portfolio = vbt.Portfolio.from_orders(price, 1000, allow_partial=True, raise_by_reject=True)
    print(portfolio.orders().records)
except Exception as e:
    print(e)

try:
    portfolio = vbt.Portfolio.from_orders(price, 1000, allow_partial=False, raise_by_reject=True)
    print(portfolio.orders().records)
except Exception as e:
    print(e)

Order rejected due to insufficient funds
Order rejected due to partial fill


In [28]:
portfolio = vbt.Portfolio.from_orders(price, order_size_one, size_type=SizeType.Shares)

print(portfolio.orders().records)

   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    1   1.0    2.0   0.0     1
2    0    3   1.0    4.0   0.0     0
3    0    4   1.0    5.0   0.0     1


In [27]:
portfolio = vbt.Portfolio.from_orders(price, 75., size_type=SizeType.TargetShares)

print(portfolio.orders().records)

   col  idx  size  price  fees  side
0    0    0  75.0    1.0   0.0     0


In [28]:
portfolio = vbt.Portfolio.from_orders(
    price_wide, 75., size_type=SizeType.TargetShares, 
    group_by=np.array([0, 0, 1]), cash_sharing=True)

print(portfolio.orders().records)

   col  idx  size  price  fees  side
0    0    0  75.0    1.0   0.0     0
1    1    0  25.0    1.0   0.0     0
2    2    0  75.0    1.0   0.0     0


In [32]:
portfolio = vbt.Portfolio.from_orders(
    price, 50., size_type=SizeType.TargetValue)

print(portfolio.orders().records)

   col  idx       size  price  fees  side
0    0    0  50.000000    1.0   0.0     0
1    0    1  25.000000    2.0   0.0     1
2    0    2   8.333333    3.0   0.0     1
3    0    3   4.166667    4.0   0.0     1
4    0    4   2.500000    5.0   0.0     1


In [4]:
portfolio = vbt.Portfolio.from_orders(
    price_wide, 50., size_type=SizeType.TargetValue,
    group_by=np.array([0, 0, 1]), cash_sharing=True)

print(portfolio.orders().records)

    col  idx       size  price  fees  side
0     0    0  50.000000    1.0   0.0     0
1     0    1  25.000000    2.0   0.0     1
2     0    2   8.333333    3.0   0.0     1
3     0    3   4.166667    4.0   0.0     1
4     0    4   2.500000    5.0   0.0     1
5     1    0  50.000000    1.0   0.0     0
6     1    1  25.000000    2.0   0.0     1
7     1    2   8.333333    3.0   0.0     1
8     1    3   4.166667    4.0   0.0     1
9     1    4   2.500000    5.0   0.0     1
10    2    0  50.000000    1.0   0.0     0
11    2    1  25.000000    2.0   0.0     1
12    2    2   8.333333    3.0   0.0     1
13    2    3   4.166667    4.0   0.0     1
14    2    4   2.500000    5.0   0.0     1


In [6]:
portfolio = vbt.Portfolio.from_orders(
    price, 0.5, size_type=SizeType.TargetPercent)

print(portfolio.orders().records)

   col  idx       size  price  fees  side
0    0    0  50.000000    1.0   0.0     0
1    0    1  12.500000    2.0   0.0     1
2    0    2   6.250000    3.0   0.0     1
3    0    3   3.906250    4.0   0.0     1
4    0    4   2.734375    5.0   0.0     1


In [8]:
portfolio = vbt.Portfolio.from_orders(
    price_wide, 0.5, size_type=SizeType.TargetPercent, 
    group_by=np.array([0, 0, 1]), cash_sharing=True)

print(portfolio.orders().records)

   col  idx       size  price  fees  side
0    0    0  50.000000    1.0   0.0     0
1    1    0  50.000000    1.0   0.0     0
2    2    0  50.000000    1.0   0.0     0
3    2    1  12.500000    2.0   0.0     1
4    2    2   6.250000    3.0   0.0     1
5    2    3   3.906250    4.0   0.0     1
6    2    4   2.734375    5.0   0.0     1


In [9]:
price_one = pd.Series([1., 1., 1., 1., 1.], index=price.index)
target_hold_value = pd.DataFrame({
    'a': [0., 70., 30., 0., 70.],
    'b': [30., 0., 70., 30., 30.],
    'c': [70., 30., 0., 70., 0.]
}, index=price.index)
print(target_hold_value)

               a     b     c
2020-01-01   0.0  30.0  70.0
2020-01-02  70.0   0.0  30.0
2020-01-03  30.0  70.0   0.0
2020-01-04   0.0  30.0  70.0
2020-01-05  70.0  30.0   0.0


In [10]:
portfolio = vbt.Portfolio.from_orders(
    price_one, target_hold_value, size_type=SizeType.TargetValue,
    group_by=np.array([0, 0, 0]), cash_sharing=True,
    call_seq=CallSeqType.Random, seed=43)

print(portfolio.orders().records)
print(portfolio.share_flow())
print(portfolio.holding_value(group_by=False))  # order fixed -> cannot rebalance properly
print(portfolio.call_seq)

   col  idx  size  price  fees  side
0    0    4  40.0    1.0   0.0     0
1    1    0  30.0    1.0   0.0     0
2    1    1  30.0    1.0   0.0     1
3    1    2  70.0    1.0   0.0     0
4    1    3  40.0    1.0   0.0     1
5    2    0  70.0    1.0   0.0     0
6    2    1  40.0    1.0   0.0     1
7    2    2  30.0    1.0   0.0     1
8    2    3  30.0    1.0   0.0     0
9    2    4  30.0    1.0   0.0     1
               a     b     c
2020-01-01   0.0  30.0  70.0
2020-01-02   0.0 -30.0 -40.0
2020-01-03   0.0  70.0 -30.0
2020-01-04   0.0 -40.0  30.0
2020-01-05  40.0   0.0 -30.0
               a     b     c
2020-01-01   0.0  30.0  70.0
2020-01-02   0.0   0.0  30.0
2020-01-03   0.0  70.0   0.0
2020-01-04   0.0  30.0  30.0
2020-01-05  40.0  30.0   0.0
            a  b  c
2020-01-01  1  2  0
2020-01-02  0  2  1
2020-01-03  1  0  2
2020-01-04  0  2  1
2020-01-05  1  0  2


In [11]:
portfolio = vbt.Portfolio.from_orders(
    price_one, target_hold_value, size_type=SizeType.TargetValue, 
    group_by=np.array([0, 0, 0]), cash_sharing=True,
    call_seq=CallSeqType.Auto)

print(portfolio.orders().records)
print(portfolio.share_flow())
print(portfolio.holding_value(group_by=False))  # order dynamic -> can rebalance
print(portfolio.call_seq)

    col  idx  size  price  fees  side
0     0    1  70.0    1.0   0.0     0
1     0    2  40.0    1.0   0.0     1
2     0    3  30.0    1.0   0.0     1
3     0    4  70.0    1.0   0.0     0
4     1    0  30.0    1.0   0.0     0
5     1    1  30.0    1.0   0.0     1
6     1    2  70.0    1.0   0.0     0
7     1    3  40.0    1.0   0.0     1
8     2    0  70.0    1.0   0.0     0
9     2    1  40.0    1.0   0.0     1
10    2    2  30.0    1.0   0.0     1
11    2    3  70.0    1.0   0.0     0
12    2    4  70.0    1.0   0.0     1
               a     b     c
2020-01-01   0.0  30.0  70.0
2020-01-02  70.0 -30.0 -40.0
2020-01-03 -40.0  70.0 -30.0
2020-01-04 -30.0 -40.0  70.0
2020-01-05  70.0   0.0 -70.0
               a     b     c
2020-01-01   0.0  30.0  70.0
2020-01-02  70.0   0.0  30.0
2020-01-03  30.0  70.0   0.0
2020-01-04   0.0  30.0  70.0
2020-01-05  70.0  30.0   0.0
            a  b  c
2020-01-01  0  1  2
2020-01-02  2  1  0
2020-01-03  0  2  1
2020-01-04  1  0  2
2020-01-05  2  1  0


In [12]:
portfolio = vbt.Portfolio.from_orders(
    price_one, target_hold_value / 100., size_type=SizeType.TargetPercent, 
    group_by=np.array([0, 0, 0]), cash_sharing=True,
    call_seq=CallSeqType.Auto)

print(portfolio.orders().records)
print(portfolio.share_flow())
print(portfolio.holding_value(group_by=False))
print(portfolio.call_seq)

    col  idx  size  price  fees  side
0     0    1  70.0    1.0   0.0     0
1     0    2  40.0    1.0   0.0     1
2     0    3  30.0    1.0   0.0     1
3     0    4  70.0    1.0   0.0     0
4     1    0  30.0    1.0   0.0     0
5     1    1  30.0    1.0   0.0     1
6     1    2  70.0    1.0   0.0     0
7     1    3  40.0    1.0   0.0     1
8     2    0  70.0    1.0   0.0     0
9     2    1  40.0    1.0   0.0     1
10    2    2  30.0    1.0   0.0     1
11    2    3  70.0    1.0   0.0     0
12    2    4  70.0    1.0   0.0     1
               a     b     c
2020-01-01   0.0  30.0  70.0
2020-01-02  70.0 -30.0 -40.0
2020-01-03 -40.0  70.0 -30.0
2020-01-04 -30.0 -40.0  70.0
2020-01-05  70.0   0.0 -70.0
               a     b     c
2020-01-01   0.0  30.0  70.0
2020-01-02  70.0   0.0  30.0
2020-01-03  30.0  70.0   0.0
2020-01-04   0.0  30.0  70.0
2020-01-05  70.0  30.0   0.0
            a  b  c
2020-01-01  0  1  2
2020-01-02  2  1  0
2020-01-03  0  2  1
2020-01-04  1  0  2
2020-01-05  2  1  0


## from_order_func

In [63]:
@njit
def order_func_nb(oc, size):
    return Order(size if oc.i % 2 == 0 else -size, SizeType.Shares, oc.close[oc.i, oc.col], 0., 0., 0., 0.)

In [64]:
%timeit vbt.Portfolio.from_order_func(\
    big_price, order_func_nb, 1., init_cash=np.inf)
%timeit vbt.Portfolio.from_order_func(\
    big_price, order_func_nb, 1., init_cash=np.inf, row_wise=True)

1.13 ms ± 455 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
943 µs ± 163 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [65]:
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_func_nb, 1., init_cash=np.inf)
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_func_nb, 1., init_cash=np.inf, row_wise=True)

95.8 ms ± 1.65 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
95.9 ms ± 7.95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [66]:
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_func_nb, 1., \
    group_by=np.repeat(np.arange(500), 2), init_cash=np.inf)
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_func_nb, 1., \
    group_by=np.repeat(np.arange(500), 2), init_cash=np.inf, row_wise=True)

82.7 ms ± 5.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
91.5 ms ± 4.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [67]:
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_func_nb, 1., \
    group_by=np.repeat(np.arange(500), 2), cash_sharing=True, init_cash=np.inf)
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_func_nb, 1., \
    group_by=np.repeat(np.arange(500), 2), cash_sharing=True, init_cash=np.inf, row_wise=True)

80.7 ms ± 4.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
91.3 ms ± 3.83 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [68]:
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_func_nb, 1., init_cash=np.inf, \
    group_by=np.full(1000, 0))
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_func_nb, 1., init_cash=np.inf, \
    group_by=np.full(1000, 0), row_wise=True)

78.4 ms ± 4.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
81.3 ms ± 4.49 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [69]:
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_func_nb, 1., init_cash=np.inf, \
    group_by=np.full(1000, 0), cash_sharing=True)
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_func_nb, 1., init_cash=np.inf, \
    group_by=np.full(1000, 0), cash_sharing=True, row_wise=True)

78 ms ± 6.45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
69.9 ms ± 8.42 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [70]:
def test_call_seq(call_seq, **kwargs):
    return vbt.Portfolio.from_order_func(
        price_wide, 
        order_func_nb, 
        1.,
        group_by=np.array([0, 0, 0]),
        call_seq=call_seq, **kwargs
    ).call_seq

def big_test_call_seq(call_seq, **kwargs):
    return vbt.Portfolio.from_order_func(
        big_price_wide, 
        order_func_nb, 
        1.,
        group_by=np.repeat(np.arange(500), 2),
        call_seq=call_seq, **kwargs
    ).call_seq

In [71]:
print(test_call_seq(CallSeqType.Default))
%timeit big_test_call_seq(CallSeqType.Default)

print(test_call_seq(CallSeqType.Reversed))
%timeit big_test_call_seq(CallSeqType.Reversed)

print(test_call_seq(CallSeqType.Random))
%timeit big_test_call_seq(CallSeqType.Random)

            a  b  c
2020-01-01  0  1  2
2020-01-02  0  1  2
2020-01-03  0  1  2
2020-01-04  0  1  2
2020-01-05  0  1  2
84.7 ms ± 4.35 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
            a  b  c
2020-01-01  2  1  0
2020-01-02  2  1  0
2020-01-03  2  1  0
2020-01-04  2  1  0
2020-01-05  2  1  0
85.1 ms ± 3.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
            a  b  c
2020-01-01  0  1  2
2020-01-02  2  1  0
2020-01-03  0  1  2
2020-01-04  0  2  1
2020-01-05  0  1  2
97.4 ms ± 4.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [72]:
@njit
def segment_prep_func1_nb(sc, *args):
    sc.call_seq_now[:] = np.arange(sc.to_col - sc.from_col - 1, -1, -1)
    return ()

@njit
def segment_prep_func2_nb(sc, *args):
    sc.call_seq_now[:] = np.copy(sc.call_seq_now[::-1])
    return ()

@njit
def segment_prep_func3_nb(sc, *args):
    call_seq_now = sc.call_seq_now
    n_cols = sc.to_col - sc.from_col
    for k in range(n_cols):
        call_seq_now[k] = n_cols - k - 1
    return ()

@njit
def segment_prep_func4_nb(sc, *args):
    np.random.shuffle(sc.call_seq_now)
    return ()

In [73]:
print(test_call_seq(CallSeqType.Default, segment_prep_func_nb=segment_prep_func1_nb))
%timeit big_test_call_seq(CallSeqType.Default, segment_prep_func_nb=segment_prep_func1_nb)
%timeit big_test_call_seq(CallSeqType.Default, segment_prep_func_nb=segment_prep_func1_nb, row_wise=True)

print(test_call_seq(CallSeqType.Default, segment_prep_func_nb=segment_prep_func2_nb))
%timeit big_test_call_seq(CallSeqType.Default, segment_prep_func_nb=segment_prep_func2_nb)
%timeit big_test_call_seq(CallSeqType.Default, segment_prep_func_nb=segment_prep_func2_nb, row_wise=True)

print(test_call_seq(CallSeqType.Default, segment_prep_func_nb=segment_prep_func3_nb))
%timeit big_test_call_seq(CallSeqType.Default, segment_prep_func_nb=segment_prep_func3_nb)
%timeit big_test_call_seq(CallSeqType.Default, segment_prep_func_nb=segment_prep_func3_nb, row_wise=True)

print(test_call_seq(CallSeqType.Default, segment_prep_func_nb=segment_prep_func4_nb))
%timeit big_test_call_seq(CallSeqType.Default, segment_prep_func_nb=segment_prep_func4_nb)  # in-place fastest
%timeit big_test_call_seq(CallSeqType.Default, segment_prep_func_nb=segment_prep_func4_nb, row_wise=True)

            a  b  c
2020-01-01  2  1  0
2020-01-02  2  1  0
2020-01-03  2  1  0
2020-01-04  2  1  0
2020-01-05  2  1  0
347 ms ± 2.61 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
367 ms ± 11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
            a  b  c
2020-01-01  2  1  0
2020-01-02  2  1  0
2020-01-03  2  1  0
2020-01-04  2  1  0
2020-01-05  2  1  0
355 ms ± 4.08 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
362 ms ± 1.66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
            a  b  c
2020-01-01  2  1  0
2020-01-02  2  1  0
2020-01-03  2  1  0
2020-01-04  2  1  0
2020-01-05  2  1  0
174 ms ± 4.83 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
193 ms ± 8.22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
            a  b  c
2020-01-01  2  0  1
2020-01-02  1  0  2
2020-01-03  2  1  0
2020-01-04  0  1  2
2020-01-05  0  2  1
179 ms ± 5.37 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
190 ms ± 4.98 ms per loop (mean ± std. dev

In [74]:
portfolio = vbt.Portfolio.from_order_func(
    price, order_func_nb, np.inf,
    target_shape=(5,))
print(portfolio.wrapper.columns)
print(portfolio.wrapper.ndim)

portfolio = vbt.Portfolio.from_order_func(
    price, order_func_nb, np.inf,
    target_shape=(5, 1))
print(portfolio.wrapper.columns)
print(portfolio.wrapper.ndim)

portfolio = vbt.Portfolio.from_order_func(
    price, order_func_nb, np.inf,
    target_shape=(5, 1), keys=pd.Index(['first'], name='custom'))
print(portfolio.wrapper.columns)
print(portfolio.wrapper.ndim)

portfolio = vbt.Portfolio.from_order_func(
    price, order_func_nb, np.inf,
    target_shape=(5, 3))
print(portfolio.wrapper.columns)
print(portfolio.wrapper.ndim)

portfolio = vbt.Portfolio.from_order_func(
    price, order_func_nb, np.inf,
    target_shape=(5, 3), keys=pd.Index(['first', 'second', 'third'], name='custom'))
print(portfolio.wrapper.columns)
print(portfolio.wrapper.ndim)

Int64Index([0], dtype='int64')
1
Int64Index([0], dtype='int64', name='iteration_idx')
2
Index(['first'], dtype='object', name='custom')
2
Int64Index([0, 1, 2], dtype='int64', name='iteration_idx')
2
Index(['first', 'second', 'third'], dtype='object', name='custom')
2


In [75]:
portfolio = vbt.Portfolio.from_order_func(price, order_func_nb, np.inf)
print(portfolio.orders().records)

portfolio = vbt.Portfolio.from_order_func(price, order_func_nb, np.inf, row_wise=True)
print(portfolio.orders().records)

   col  idx        size  price  fees  side
0    0    0  100.000000    1.0   0.0     0
1    0    1  100.000000    2.0   0.0     1
2    0    2   66.666667    3.0   0.0     0
3    0    3   66.666667    4.0   0.0     1
4    0    4   53.333333    5.0   0.0     0
   col  idx        size  price  fees  side
0    0    0  100.000000    1.0   0.0     0
1    0    1  100.000000    2.0   0.0     1
2    0    2   66.666667    3.0   0.0     0
3    0    3   66.666667    4.0   0.0     1
4    0    4   53.333333    5.0   0.0     0


In [76]:
portfolio = vbt.Portfolio.from_order_func(price_wide, order_func_nb, np.inf)
print(portfolio.orders().records)

portfolio = vbt.Portfolio.from_order_func(price_wide, order_func_nb, np.inf, row_wise=True)
print(portfolio.orders().records)

    col  idx        size  price  fees  side
0     0    0  100.000000    1.0   0.0     0
1     0    1  100.000000    2.0   0.0     1
2     0    2   66.666667    3.0   0.0     0
3     0    3   66.666667    4.0   0.0     1
4     0    4   53.333333    5.0   0.0     0
5     1    0  100.000000    1.0   0.0     0
6     1    1  100.000000    2.0   0.0     1
7     1    2   66.666667    3.0   0.0     0
8     1    3   66.666667    4.0   0.0     1
9     1    4   53.333333    5.0   0.0     0
10    2    0  100.000000    1.0   0.0     0
11    2    1  100.000000    2.0   0.0     1
12    2    2   66.666667    3.0   0.0     0
13    2    3   66.666667    4.0   0.0     1
14    2    4   53.333333    5.0   0.0     0
    col  idx        size  price  fees  side
0     0    0  100.000000    1.0   0.0     0
1     0    1  100.000000    2.0   0.0     1
2     0    2   66.666667    3.0   0.0     0
3     0    3   66.666667    4.0   0.0     1
4     0    4   53.333333    5.0   0.0     0
5     1    0  100.000000    1.0 

In [77]:
portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_func_nb, np.inf,
    group_by=np.array([0, 0, 1]))
print(portfolio.orders().records)

portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_func_nb, np.inf,
    group_by=np.array([0, 0, 1]), row_wise=True)
print(portfolio.orders().records)

    col  idx        size  price  fees  side
0     0    0  100.000000    1.0   0.0     0
1     0    1  100.000000    2.0   0.0     1
2     0    2   66.666667    3.0   0.0     0
3     0    3   66.666667    4.0   0.0     1
4     0    4   53.333333    5.0   0.0     0
5     1    0  100.000000    1.0   0.0     0
6     1    1  100.000000    2.0   0.0     1
7     1    2   66.666667    3.0   0.0     0
8     1    3   66.666667    4.0   0.0     1
9     1    4   53.333333    5.0   0.0     0
10    2    0  100.000000    1.0   0.0     0
11    2    1  100.000000    2.0   0.0     1
12    2    2   66.666667    3.0   0.0     0
13    2    3   66.666667    4.0   0.0     1
14    2    4   53.333333    5.0   0.0     0
    col  idx        size  price  fees  side
0     0    0  100.000000    1.0   0.0     0
1     0    1  100.000000    2.0   0.0     1
2     0    2   66.666667    3.0   0.0     0
3     0    3   66.666667    4.0   0.0     1
4     0    4   53.333333    5.0   0.0     0
5     1    0  100.000000    1.0 

In [78]:
portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_func_nb, np.inf,
    group_by=np.array([0, 0, 1]), cash_sharing=True)
print(portfolio.orders().records)

portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_func_nb, np.inf,
    group_by=np.array([0, 0, 1]), cash_sharing=True, row_wise=True)
print(portfolio.orders().records)

   col  idx        size  price  fees  side
0    0    0  100.000000    1.0   0.0     0
1    0    1  100.000000    2.0   0.0     1
2    0    2   66.666667    3.0   0.0     0
3    0    3   66.666667    4.0   0.0     1
4    0    4   53.333333    5.0   0.0     0
5    2    0  100.000000    1.0   0.0     0
6    2    1  100.000000    2.0   0.0     1
7    2    2   66.666667    3.0   0.0     0
8    2    3   66.666667    4.0   0.0     1
9    2    4   53.333333    5.0   0.0     0
   col  idx        size  price  fees  side
0    0    0  100.000000    1.0   0.0     0
1    0    1  100.000000    2.0   0.0     1
2    0    2   66.666667    3.0   0.0     0
3    0    3   66.666667    4.0   0.0     1
4    0    4   53.333333    5.0   0.0     0
5    2    0  100.000000    1.0   0.0     0
6    2    1  100.000000    2.0   0.0     1
7    2    2   66.666667    3.0   0.0     0
8    2    3   66.666667    4.0   0.0     1
9    2    4   53.333333    5.0   0.0     0


In [79]:
@njit
def target_val_segment_prep_func_nb(sc, val_price):
    sc.last_val_price[sc.from_col:sc.to_col] = val_price[sc.i]
    return ()

@njit
def target_val_order_func_nb(oc):
    print(oc.last_val_price[oc.from_col:oc.to_col], 50. / oc.close[oc.i, oc.col] - oc.shares_now)
    return Order(50., SizeType.TargetValue, oc.close[oc.i, oc.col], 0., 0., 0., 0.)

portfolio = vbt.Portfolio.from_order_func(
    price.iloc[1:], target_val_order_func_nb, 
    segment_prep_func_nb=target_val_segment_prep_func_nb,
    segment_prep_args=(price.iloc[:-1],)  # lagged valuation price
)
print(portfolio.orders().records)

portfolio = vbt.Portfolio.from_order_func(
    price.iloc[1:], target_val_order_func_nb, 
    segment_prep_func_nb=target_val_segment_prep_func_nb,
    segment_prep_args=(price.iloc[:-1],), row_wise=True
)
print(portfolio.orders().records)

[1.] 25.0
[2.] -33.33333333333333
[3.] -12.5
[4.] -6.666666666666668
   col  idx       size  price  fees  side
0    0    0  50.000000    2.0   0.0     0
1    0    1  25.000000    3.0   0.0     1
2    0    2   8.333333    4.0   0.0     1
3    0    3   4.166667    5.0   0.0     1
[1.] 25.0
[2.] -33.33333333333333
[3.] -12.5
[4.] -6.666666666666668
   col  idx       size  price  fees  side
0    0    0  50.000000    2.0   0.0     0
1    0    1  25.000000    3.0   0.0     1
2    0    2   8.333333    4.0   0.0     1
3    0    3   4.166667    5.0   0.0     1


In [80]:
portfolio = vbt.Portfolio.from_order_func(
    price.iloc[1:], target_val_order_func_nb, 
    segment_prep_func_nb=target_val_segment_prep_func_nb,
    segment_prep_args=(price.iloc[1:],)  # current order price
)
print(portfolio.orders().records)

portfolio = vbt.Portfolio.from_order_func(
    price.iloc[1:], target_val_order_func_nb, 
    segment_prep_func_nb=target_val_segment_prep_func_nb,
    segment_prep_args=(price.iloc[1:],), row_wise=True
)
print(portfolio.orders().records)

[2.] 25.0
[3.] -8.333333333333332
[4.] -4.166666666666668
[5.] -2.5
   col  idx       size  price  fees  side
0    0    0  25.000000    2.0   0.0     0
1    0    1   8.333333    3.0   0.0     1
2    0    2   4.166667    4.0   0.0     1
3    0    3   2.500000    5.0   0.0     1
[2.] 25.0
[3.] -8.333333333333332
[4.] -4.166666666666668
[5.] -2.5
   col  idx       size  price  fees  side
0    0    0  25.000000    2.0   0.0     0
1    0    1   8.333333    3.0   0.0     1
2    0    2   4.166667    4.0   0.0     1
3    0    3   2.500000    5.0   0.0     1


In [81]:
@njit
def target_pct_segment_prep_func_nb(sc, val_price):
    sc.last_val_price[sc.from_col:sc.to_col] = val_price[sc.i]
    return ()

@njit
def target_pct_order_func_nb(oc):
    print(oc.last_val_price[oc.from_col:oc.to_col], 0.5 * oc.value_now / oc.close[oc.i, oc.col] - oc.shares_now)
    return Order(0.5, SizeType.TargetPercent, oc.close[oc.i, oc.col], 0., 0., 0., 0.)

portfolio = vbt.Portfolio.from_order_func(
    price.iloc[1:], target_pct_order_func_nb, 
    segment_prep_func_nb=target_pct_segment_prep_func_nb,
    segment_prep_args=(price.iloc[:-1],)  # lagged valuation price
)
print(portfolio.orders().records)

portfolio = vbt.Portfolio.from_order_func(
    price.iloc[1:], target_pct_order_func_nb, 
    segment_prep_func_nb=target_pct_segment_prep_func_nb,
    segment_prep_args=(price.iloc[:-1],), row_wise=True
)
print(portfolio.orders().records)

[1.] 25.0
[2.] -33.33333333333333
[3.] -6.25
[4.] -7.5
   col  idx    size  price  fees  side
0    0    0  50.000    2.0   0.0     0
1    0    1  25.000    3.0   0.0     1
2    0    3   3.125    5.0   0.0     1
[1.] 25.0
[2.] -33.33333333333333
[3.] -6.25
[4.] -7.5
   col  idx    size  price  fees  side
0    0    0  50.000    2.0   0.0     0
1    0    1  25.000    3.0   0.0     1
2    0    3   3.125    5.0   0.0     1


In [82]:
portfolio = vbt.Portfolio.from_order_func(
    price.iloc[1:], target_pct_order_func_nb, 
    segment_prep_func_nb=target_pct_segment_prep_func_nb,
    segment_prep_args=(price.iloc[1:],)  # current order price
)
print(portfolio.orders().records)

portfolio = vbt.Portfolio.from_order_func(
    price.iloc[1:], target_pct_order_func_nb, 
    segment_prep_func_nb=target_pct_segment_prep_func_nb,
    segment_prep_args=(price.iloc[1:],), row_wise=True
)
print(portfolio.orders().records)

[2.] 25.0
[3.] -4.166666666666668
[4.] -2.604166666666668
[5.] -1.8229166666666643
   col  idx       size  price  fees  side
0    0    0  25.000000    2.0   0.0     0
1    0    1   4.166667    3.0   0.0     1
2    0    2   2.604167    4.0   0.0     1
3    0    3   1.822917    5.0   0.0     1
[2.] 25.0
[3.] -4.166666666666668
[4.] -2.604166666666668
[5.] -1.8229166666666643
   col  idx       size  price  fees  side
0    0    0  25.000000    2.0   0.0     0
1    0    1   4.166667    3.0   0.0     1
2    0    2   2.604167    4.0   0.0     1
3    0    3   1.822917    5.0   0.0     1


In [83]:
portfolio = vbt.Portfolio.from_order_func(
    price, order_func_nb, 1.,
    active_mask=pd.Series([True, False, True, False, True]))
print(portfolio.orders().records)

portfolio = vbt.Portfolio.from_order_func(
    price, order_func_nb, 1.,
    active_mask=pd.Series([True, False, True, False, True]), row_wise=True)
print(portfolio.orders().records)

   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    2   1.0    3.0   0.0     0
2    0    4   1.0    5.0   0.0     0
   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    2   1.0    3.0   0.0     0
2    0    4   1.0    5.0   0.0     0


In [84]:
@njit
def prep_func_nb(simc, sim_i):
    print('preparing simulation')
    sim_i[0] += 1
    return sim_i,

@njit
def group_prep_func_nb(gc, sim_i, group_i):
    print('\tpreparing group', gc.group)
    group_i[0] += 1
    return sim_i, group_i

@njit
def segment_prep_func_nb(sc, sim_i, group_i, segment_i):
    print('\t\tpreparing segment', sc.i, '(row)')
    segment_i[0] += 1
    return sim_i, group_i, segment_i

@njit
def order_func_nb(oc, sim_i, group_i, segment_i, order_i):
    print('\t\t\trunning order', oc.call_idx, 'at column', oc.col)
    order_i[0] += 1
    return NoOrder

sim_i = np.array([0])
group_i = np.array([0])
segment_i = np.array([0])
order_i = np.array([0])
portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_func_nb, order_i, 
    group_by=np.array([0, 0, 1]), 
    prep_func_nb=prep_func_nb, prep_args=(sim_i,),
    group_prep_func_nb=group_prep_func_nb, group_prep_args=(group_i,),
    segment_prep_func_nb=segment_prep_func_nb, segment_prep_args=(segment_i,)
)
print(sim_i, group_i, segment_i, order_i)

preparing simulation
	preparing group 0
		preparing segment 0 (row)
			running order 0 at column 0
			running order 1 at column 1
		preparing segment 1 (row)
			running order 0 at column 0
			running order 1 at column 1
		preparing segment 2 (row)
			running order 0 at column 0
			running order 1 at column 1
		preparing segment 3 (row)
			running order 0 at column 0
			running order 1 at column 1
		preparing segment 4 (row)
			running order 0 at column 0
			running order 1 at column 1
	preparing group 1
		preparing segment 0 (row)
			running order 0 at column 2
		preparing segment 1 (row)
			running order 0 at column 2
		preparing segment 2 (row)
			running order 0 at column 2
		preparing segment 3 (row)
			running order 0 at column 2
		preparing segment 4 (row)
			running order 0 at column 2
[1] [2] [10] [15]


In [85]:
@njit
def prep_func_nb(simc, sim_i):
    print('preparing simulation')
    sim_i[0] += 1
    return sim_i,

@njit
def row_prep_func_nb(gc, sim_i, row_i):
    print('\tpreparing row', gc.i)
    row_i[0] += 1
    return sim_i, row_i

@njit
def segment_prep_func_nb(sc, sim_i, row_i, segment_i):
    print('\t\tpreparing segment', sc.group, '(group)')
    segment_i[0] += 1
    return sim_i, row_i, segment_i

@njit
def order_func_nb(oc, sim_i, row_i, segment_i, order_i):
    print('\t\t\trunning order', oc.call_idx, 'at column', oc.col)
    order_i[0] += 1
    return NoOrder

sim_i = np.array([0])
row_i = np.array([0])
segment_i = np.array([0])
order_i = np.array([0])

portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_func_nb, order_i, 
    group_by=np.array([0, 0, 1]), 
    prep_func_nb=prep_func_nb, prep_args=(sim_i,),
    row_prep_func_nb=row_prep_func_nb, row_prep_args=(row_i,),
    segment_prep_func_nb=segment_prep_func_nb, segment_prep_args=(segment_i,),
    row_wise=True
)
print(sim_i, row_i, segment_i, order_i)

preparing simulation
	preparing row 0
		preparing segment 0 (group)
			running order 0 at column 0
			running order 1 at column 1
		preparing segment 1 (group)
			running order 0 at column 2
	preparing row 1
		preparing segment 0 (group)
			running order 0 at column 0
			running order 1 at column 1
		preparing segment 1 (group)
			running order 0 at column 2
	preparing row 2
		preparing segment 0 (group)
			running order 0 at column 0
			running order 1 at column 1
		preparing segment 1 (group)
			running order 0 at column 2
	preparing row 3
		preparing segment 0 (group)
			running order 0 at column 0
			running order 1 at column 1
		preparing segment 1 (group)
			running order 0 at column 2
	preparing row 4
		preparing segment 0 (group)
			running order 0 at column 0
			running order 1 at column 1
		preparing segment 1 (group)
			running order 0 at column 2
[1] [5] [10] [15]


## methods and properties

In [86]:
price_na = pd.DataFrame({
    'a': [np.nan, 2., 3., 4., 5.],
    'b': [1., 2., np.nan, 4., 5.],
    'c': [1., 2., 3., 4., np.nan]
}, index=price.index)
order_size = pd.Series([1., 0.1, -1., -0.1, 1.])

In [88]:
portfolio = vbt.Portfolio.from_orders(
    price_na, order_size, size_type=SizeType.Shares, 
    fees=0.01, fixed_fees=0.1, slippage=0.01,
    init_cash=[100., 100., 100.],
    call_seq=CallSeqType.Reversed,
    group_by=None
)  # independent
print(portfolio.orders().records)

group_by = np.array(['first', 'first', 'second'])
portfolio_grouped = vbt.Portfolio.from_orders(
    price_na, order_size, size_type=SizeType.Shares, 
    fees=0.01, fixed_fees=0.1, slippage=0.01,
    init_cash=[100., 100., 100.],
    call_seq=CallSeqType.Reversed,
    group_by=group_by,
    cash_sharing=False
)  # grouped
print(portfolio_grouped.orders().records)

portfolio_shared = vbt.Portfolio.from_orders(
    price_na, order_size, size_type=SizeType.Shares, 
    fees=0.01, fixed_fees=0.1, slippage=0.01,
    init_cash=[200., 100.],
    call_seq=CallSeqType.Reversed,
    group_by=group_by,
    cash_sharing=True
)  # shared
print(portfolio_shared.orders().records)

    col  idx  size  price     fees  side
0     0    1   0.1   2.02  0.10202     0
1     0    2   0.1   2.97  0.10297     1
2     0    4   1.0   5.05  0.15050     0
3     1    0   1.0   1.01  0.11010     0
4     1    1   0.1   2.02  0.10202     0
5     1    3   0.1   3.96  0.10396     1
6     1    4   1.0   5.05  0.15050     0
7     2    0   1.0   1.01  0.11010     0
8     2    1   0.1   2.02  0.10202     0
9     2    2   1.0   2.97  0.12970     1
10    2    3   0.1   3.96  0.10396     1
    col  idx  size  price     fees  side
0     0    1   0.1   2.02  0.10202     0
1     0    2   0.1   2.97  0.10297     1
2     0    4   1.0   5.05  0.15050     0
3     1    0   1.0   1.01  0.11010     0
4     1    1   0.1   2.02  0.10202     0
5     1    3   0.1   3.96  0.10396     1
6     1    4   1.0   5.05  0.15050     0
7     2    0   1.0   1.01  0.11010     0
8     2    1   0.1   2.02  0.10202     0
9     2    2   1.0   2.97  0.12970     1
10    2    3   0.1   3.96  0.10396     1
    col  idx  si

In [90]:
big_group_by = np.repeat(np.arange(500), 2)
big_portfolio = vbt.Portfolio.from_orders(
    big_price_wide, big_order_size)
print(big_portfolio.orders().records_arr.shape)

big_portfolio_grouped = vbt.Portfolio.from_orders(
    big_price_wide, big_order_size, group_by=big_group_by)
print(big_portfolio_grouped.orders().records_arr.shape)

big_portfolio_shared = vbt.Portfolio.from_orders(
    big_price_wide, big_order_size, group_by=big_group_by, cash_sharing=True)
print(big_portfolio_shared.orders().records_arr.shape)

(1000000,)
(1000000,)
(1000000,)


In [95]:
print(portfolio.orders().records)
print(portfolio.orders().count())
print(portfolio.close)
print(portfolio.init_cash())
print(portfolio.value())
print(portfolio.final_value())

    col  idx  size  price     fees  side
0     0    1   0.1   2.02  0.10202     0
1     0    2   0.1   2.97  0.10297     1
2     0    4   1.0   5.05  0.15050     0
3     1    0   1.0   1.01  0.11010     0
4     1    1   0.1   2.02  0.10202     0
5     1    3   0.1   3.96  0.10396     1
6     1    4   1.0   5.05  0.15050     0
7     2    0   1.0   1.01  0.11010     0
8     2    1   0.1   2.02  0.10202     0
9     2    2   1.0   2.97  0.12970     1
10    2    3   0.1   3.96  0.10396     1
a    3
b    4
c    4
dtype: int64
              a    b    c
2020-01-01  NaN  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  NaN  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  NaN
a    100.0
b    100.0
c    100.0
dtype: float64
                    a          b          c
2020-01-01  100.00000   99.87990   99.87990
2020-01-02   99.89598  100.77588  100.77588
2020-01-03   99.89001        NaN  101.71618
2020-01-04   99.89001  102.86792  101.70822
2020-01-05   99.68951  103.66742  101.70822
a     

In [96]:
print(portfolio['a'].orders().records)
print(portfolio['a'].orders().count())
print(portfolio['a'].close)
print(portfolio['a'].init_cash())
print(portfolio['a'].value())
print(portfolio['a'].final_value())

   col  idx  size  price     fees  side
0    0    1   0.1   2.02  0.10202     0
1    0    2   0.1   2.97  0.10297     1
2    0    4   1.0   5.05  0.15050     0
3
2020-01-01    NaN
2020-01-02    2.0
2020-01-03    3.0
2020-01-04    4.0
2020-01-05    5.0
Name: a, dtype: float64
100.0
2020-01-01    100.00000
2020-01-02     99.89598
2020-01-03     99.89001
2020-01-04     99.89001
2020-01-05     99.68951
Name: a, dtype: float64
99.68951


In [97]:
print(portfolio['c'].orders().records)
print(portfolio['c'].orders().count())
print(portfolio['c'].close)
print(portfolio['c'].init_cash())
print(portfolio['c'].value())
print(portfolio['c'].final_value())

   col  idx  size  price     fees  side
0    0    0   1.0   1.01  0.11010     0
1    0    1   0.1   2.02  0.10202     0
2    0    2   1.0   2.97  0.12970     1
3    0    3   0.1   3.96  0.10396     1
4
2020-01-01    1.0
2020-01-02    2.0
2020-01-03    3.0
2020-01-04    4.0
2020-01-05    NaN
Name: c, dtype: float64
100.0
2020-01-01     99.87990
2020-01-02    100.77588
2020-01-03    101.71618
2020-01-04    101.70822
2020-01-05    101.70822
Name: c, dtype: float64
101.70822000000001


In [98]:
print(portfolio[['c']].orders().records)
print(portfolio[['c']].orders().count())
print(portfolio[['c']].close)
print(portfolio[['c']].init_cash())
print(portfolio[['c']].value())
print(portfolio[['c']].final_value())

   col  idx  size  price     fees  side
0    0    0   1.0   1.01  0.11010     0
1    0    1   0.1   2.02  0.10202     0
2    0    2   1.0   2.97  0.12970     1
3    0    3   0.1   3.96  0.10396     1
c    4
dtype: int64
              c
2020-01-01  1.0
2020-01-02  2.0
2020-01-03  3.0
2020-01-04  4.0
2020-01-05  NaN
c    100.0
dtype: float64
                    c
2020-01-01   99.87990
2020-01-02  100.77588
2020-01-03  101.71618
2020-01-04  101.70822
2020-01-05  101.70822
c    101.70822
dtype: float64


In [99]:
print(portfolio_grouped.orders().records)
print(portfolio_grouped.orders().count())
print(portfolio_grouped.close)
print(portfolio_grouped.init_cash())
print(portfolio_grouped.value())
print(portfolio_grouped.final_value())

    col  idx  size  price     fees  side
0     0    1   0.1   2.02  0.10202     0
1     0    2   0.1   2.97  0.10297     1
2     0    4   1.0   5.05  0.15050     0
3     1    0   1.0   1.01  0.11010     0
4     1    1   0.1   2.02  0.10202     0
5     1    3   0.1   3.96  0.10396     1
6     1    4   1.0   5.05  0.15050     0
7     2    0   1.0   1.01  0.11010     0
8     2    1   0.1   2.02  0.10202     0
9     2    2   1.0   2.97  0.12970     1
10    2    3   0.1   3.96  0.10396     1
first     7
second    4
dtype: int64
              a    b    c
2020-01-01  NaN  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  NaN  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  NaN
first     200.0
second    100.0
dtype: float64
                first     second
2020-01-01  199.87990   99.87990
2020-01-02  200.67186  100.77588
2020-01-03        NaN  101.71618
2020-01-04  202.75793  101.70822
2020-01-05  203.35693  101.70822
first     203.35693
second    101.70822
dtype: float64


In [100]:
print(portfolio_grouped['first'].orders().records)
print(portfolio_grouped['first'].orders().count())
print(portfolio_grouped['first'].close)
print(portfolio_grouped['first'].init_cash())
print(portfolio_grouped['first'].value())
print(portfolio_grouped['first'].final_value())

   col  idx  size  price     fees  side
0    0    1   0.1   2.02  0.10202     0
1    0    2   0.1   2.97  0.10297     1
2    0    4   1.0   5.05  0.15050     0
3    1    0   1.0   1.01  0.11010     0
4    1    1   0.1   2.02  0.10202     0
5    1    3   0.1   3.96  0.10396     1
6    1    4   1.0   5.05  0.15050     0
7
              a    b
2020-01-01  NaN  1.0
2020-01-02  2.0  2.0
2020-01-03  3.0  NaN
2020-01-04  4.0  4.0
2020-01-05  5.0  5.0
200.0
2020-01-01    199.87990
2020-01-02    200.67186
2020-01-03          NaN
2020-01-04    202.75793
2020-01-05    203.35693
Name: first, dtype: float64
203.35693


In [101]:
print(portfolio_grouped[['first']].orders().records)  # same because of collapse_group
print(portfolio_grouped[['first']].orders().count())
print(portfolio_grouped[['first']].close)
print(portfolio_grouped[['first']].init_cash())
print(portfolio_grouped[['first']].value())
print(portfolio_grouped[['first']].final_value())

   col  idx  size  price     fees  side
0    0    1   0.1   2.02  0.10202     0
1    0    2   0.1   2.97  0.10297     1
2    0    4   1.0   5.05  0.15050     0
3    1    0   1.0   1.01  0.11010     0
4    1    1   0.1   2.02  0.10202     0
5    1    3   0.1   3.96  0.10396     1
6    1    4   1.0   5.05  0.15050     0
first    7
dtype: int64
              a    b
2020-01-01  NaN  1.0
2020-01-02  2.0  2.0
2020-01-03  3.0  NaN
2020-01-04  4.0  4.0
2020-01-05  5.0  5.0
first    200.0
dtype: float64
                first
2020-01-01  199.87990
2020-01-02  200.67186
2020-01-03        NaN
2020-01-04  202.75793
2020-01-05  203.35693
first    203.35693
dtype: float64


In [102]:
print(portfolio_grouped['second'].orders().records)
print(portfolio_grouped['second'].orders().count())
print(portfolio_grouped['second'].close)
print(portfolio_grouped['second'].init_cash())
print(portfolio_grouped['second'].value())
print(portfolio_grouped['second'].final_value())

   col  idx  size  price     fees  side
0    0    0   1.0   1.01  0.11010     0
1    0    1   0.1   2.02  0.10202     0
2    0    2   1.0   2.97  0.12970     1
3    0    3   0.1   3.96  0.10396     1
4
2020-01-01    1.0
2020-01-02    2.0
2020-01-03    3.0
2020-01-04    4.0
2020-01-05    NaN
Name: c, dtype: float64
100.0
2020-01-01     99.87990
2020-01-02    100.77588
2020-01-03    101.71618
2020-01-04    101.70822
2020-01-05    101.70822
Name: second, dtype: float64
101.70822000000001


In [103]:
print(portfolio_grouped[['second']].orders().records)
print(portfolio_grouped[['second']].orders().count())
print(portfolio_grouped[['second']].close)
print(portfolio_grouped[['second']].init_cash())
print(portfolio_grouped[['second']].value())
print(portfolio_grouped[['second']].final_value())

   col  idx  size  price     fees  side
0    0    0   1.0   1.01  0.11010     0
1    0    1   0.1   2.02  0.10202     0
2    0    2   1.0   2.97  0.12970     1
3    0    3   0.1   3.96  0.10396     1
second    4
dtype: int64
              c
2020-01-01  1.0
2020-01-02  2.0
2020-01-03  3.0
2020-01-04  4.0
2020-01-05  NaN
second    100.0
dtype: float64
               second
2020-01-01   99.87990
2020-01-02  100.77588
2020-01-03  101.71618
2020-01-04  101.70822
2020-01-05  101.70822
second    101.70822
dtype: float64


In [104]:
print(portfolio_shared['first'].orders().records)
print(portfolio_shared['first'].orders().count())
print(portfolio_shared['first'].close)
print(portfolio_shared['first'].init_cash())
print(portfolio_shared['first'].value())
print(portfolio_shared['first'].final_value())

   col  idx  size  price     fees  side
0    0    1   0.1   2.02  0.10202     0
1    0    2   0.1   2.97  0.10297     1
2    0    4   1.0   5.05  0.15050     0
3    1    0   1.0   1.01  0.11010     0
4    1    1   0.1   2.02  0.10202     0
5    1    3   0.1   3.96  0.10396     1
6    1    4   1.0   5.05  0.15050     0
7
              a    b
2020-01-01  NaN  1.0
2020-01-02  2.0  2.0
2020-01-03  3.0  NaN
2020-01-04  4.0  4.0
2020-01-05  5.0  5.0
200.0
2020-01-01    199.87990
2020-01-02    200.67186
2020-01-03          NaN
2020-01-04    202.75793
2020-01-05    203.35693
Name: first, dtype: float64
203.35692999999995


In [105]:
print(portfolio_shared[['first']].orders().records)
print(portfolio_shared[['first']].orders().count())
print(portfolio_shared[['first']].close)
print(portfolio_shared[['first']].init_cash())
print(portfolio_shared[['first']].value())
print(portfolio_shared[['first']].final_value())

   col  idx  size  price     fees  side
0    0    1   0.1   2.02  0.10202     0
1    0    2   0.1   2.97  0.10297     1
2    0    4   1.0   5.05  0.15050     0
3    1    0   1.0   1.01  0.11010     0
4    1    1   0.1   2.02  0.10202     0
5    1    3   0.1   3.96  0.10396     1
6    1    4   1.0   5.05  0.15050     0
first    7
dtype: int64
              a    b
2020-01-01  NaN  1.0
2020-01-02  2.0  2.0
2020-01-03  3.0  NaN
2020-01-04  4.0  4.0
2020-01-05  5.0  5.0
first    200.0
dtype: float64
                first
2020-01-01  199.87990
2020-01-02  200.67186
2020-01-03        NaN
2020-01-04  202.75793
2020-01-05  203.35693
first    203.35693
dtype: float64


In [106]:
print(portfolio_shared['second'].orders().records)
print(portfolio_shared['second'].orders().count())
print(portfolio_shared['second'].close)
print(portfolio_shared['second'].init_cash())
print(portfolio_shared['second'].value())
print(portfolio_shared['second'].final_value())

   col  idx  size  price     fees  side
0    0    0   1.0   1.01  0.11010     0
1    0    1   0.1   2.02  0.10202     0
2    0    2   1.0   2.97  0.12970     1
3    0    3   0.1   3.96  0.10396     1
4
2020-01-01    1.0
2020-01-02    2.0
2020-01-03    3.0
2020-01-04    4.0
2020-01-05    NaN
Name: c, dtype: float64
100.0
2020-01-01     99.87990
2020-01-02    100.77588
2020-01-03    101.71618
2020-01-04    101.70822
2020-01-05    101.70822
Name: second, dtype: float64
101.70822000000001


In [107]:
print(portfolio_shared[['second']].orders().records)
print(portfolio_shared[['second']].orders().count())
print(portfolio_shared[['second']].close)
print(portfolio_shared[['second']].init_cash())
print(portfolio_shared[['second']].value())
print(portfolio_shared[['second']].final_value())

   col  idx  size  price     fees  side
0    0    0   1.0   1.01  0.11010     0
1    0    1   0.1   2.02  0.10202     0
2    0    2   1.0   2.97  0.12970     1
3    0    3   0.1   3.96  0.10396     1
second    4
dtype: int64
              c
2020-01-01  1.0
2020-01-02  2.0
2020-01-03  3.0
2020-01-04  4.0
2020-01-05  NaN
second    100.0
dtype: float64
               second
2020-01-01   99.87990
2020-01-02  100.77588
2020-01-03  101.71618
2020-01-04  101.70822
2020-01-05  101.70822
second    101.70822
dtype: float64


In [108]:
%timeit big_portfolio.iloc[0]
%timeit big_portfolio.iloc[:]
%timeit big_portfolio_grouped.iloc[0]
%timeit big_portfolio_grouped.iloc[:]
%timeit big_portfolio_shared.iloc[0]
%timeit big_portfolio_shared.iloc[:]

6.54 ms ± 1.37 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
39.3 ms ± 6.93 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
4.81 ms ± 62.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
32.8 ms ± 251 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
5.08 ms ± 150 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
36.2 ms ± 2.85 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [109]:
print(portfolio.cash_sharing)
print(portfolio_grouped.cash_sharing)
print(portfolio_shared.cash_sharing)

False
False
True


In [110]:
print(portfolio.call_seq)
print(portfolio_grouped.call_seq)
print(portfolio_shared.call_seq)

            a  b  c
2020-01-01  0  0  0
2020-01-02  0  0  0
2020-01-03  0  0  0
2020-01-04  0  0  0
2020-01-05  0  0  0
            a  b  c
2020-01-01  1  0  0
2020-01-02  1  0  0
2020-01-03  1  0  0
2020-01-04  1  0  0
2020-01-05  1  0  0
            a  b  c
2020-01-01  1  0  0
2020-01-02  1  0  0
2020-01-03  1  0  0
2020-01-04  1  0  0
2020-01-05  1  0  0


In [111]:
print(portfolio.regroup(group_by).value())

                first     second
2020-01-01  199.87990   99.87990
2020-01-02  200.67186  100.77588
2020-01-03        NaN  101.71618
2020-01-04  202.75793  101.70822
2020-01-05  203.35693  101.70822


In [112]:
print(portfolio.close)
print(portfolio_grouped.close)
print(portfolio_shared.close)

              a    b    c
2020-01-01  NaN  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  NaN  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  NaN
              a    b    c
2020-01-01  NaN  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  NaN  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  NaN
              a    b    c
2020-01-01  NaN  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  NaN  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  NaN


In [113]:
print(portfolio.fill_close(ffill=False, bfill=False))
print(portfolio.fill_close(ffill=True, bfill=False))
print(portfolio.fill_close(ffill=False, bfill=True))
print(portfolio.fill_close(ffill=True, bfill=True))

              a    b    c
2020-01-01  NaN  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  NaN  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  NaN
              a    b    c
2020-01-01  NaN  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  2.0  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  4.0
              a    b    c
2020-01-01  2.0  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  4.0  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  NaN
              a    b    c
2020-01-01  2.0  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  2.0  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  4.0


In [114]:
%timeit big_portfolio.fill_close()

179 µs ± 17.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [116]:
print(vbt.Portfolio.from_orders(
    price_na, 1000., init_cash=InitCashMode.Auto, group_by=None).init_cash())

print(vbt.Portfolio.from_orders(
    price_na, 1000., init_cash=InitCashMode.Auto, group_by=group_by).init_cash())

print(vbt.Portfolio.from_orders(
    price_na, 1000., init_cash=InitCashMode.Auto, group_by=group_by, cash_sharing=True).init_cash())

print(vbt.Portfolio.from_orders(
    price_na, 1000., init_cash=InitCashMode.AutoAlign, group_by=None).init_cash())

print(vbt.Portfolio.from_orders(
    price_na, 1000., init_cash=InitCashMode.AutoAlign, group_by=group_by).init_cash())

print(vbt.Portfolio.from_orders(
    price_na, 1000., init_cash=InitCashMode.AutoAlign, group_by=group_by, cash_sharing=True).init_cash())

a    14000.0
b    12000.0
c    10000.0
dtype: float64
first     26000.0
second    10000.0
dtype: float64
first     26000.0
second    10000.0
dtype: float64
a    14000.0
b    14000.0
c    14000.0
dtype: float64
first     26000.0
second    26000.0
dtype: float64
first     26000.0
second    26000.0
dtype: float64


In [118]:
print(portfolio.init_cash())
print(portfolio_grouped.init_cash(group_by=False))
print(portfolio_shared.init_cash(group_by=False))

print(portfolio.init_cash(group_by=group_by))
print(portfolio_grouped.init_cash())
print(portfolio_shared.init_cash())

a    100.0
b    100.0
c    100.0
dtype: float64
a    100.0
b    100.0
c    100.0
dtype: float64
a    200.0
b    200.0
c    100.0
dtype: float64
first     200.0
second    100.0
dtype: float64
first     200.0
second    100.0
dtype: float64
first     200.0
second    100.0
dtype: float64


In [119]:
%timeit big_portfolio.init_cash()
%timeit big_portfolio_grouped.init_cash(group_by=False)
%timeit big_portfolio_shared.init_cash(group_by=False)

%timeit big_portfolio.init_cash(group_by=big_group_by)
%timeit big_portfolio_grouped.init_cash()
%timeit big_portfolio_shared.init_cash()

189 µs ± 63.4 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
415 µs ± 11.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
572 µs ± 113 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
565 µs ± 13.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
586 µs ± 81.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
573 µs ± 17.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [120]:
print(portfolio.cash_flow())
print(portfolio_grouped.cash_flow(group_by=False))
print(portfolio_shared.cash_flow(group_by=False))

print(portfolio.cash_flow(group_by=group_by))
print(portfolio_grouped.cash_flow())
print(portfolio_shared.cash_flow())

                  a        b        c
2020-01-01  0.00000 -1.12010 -1.12010
2020-01-02 -0.30402 -0.30402 -0.30402
2020-01-03  0.19403  0.00000  2.84030
2020-01-04  0.00000  0.29204  0.29204
2020-01-05 -5.20050 -5.20050  0.00000
                  a        b        c
2020-01-01  0.00000 -1.12010 -1.12010
2020-01-02 -0.30402 -0.30402 -0.30402
2020-01-03  0.19403  0.00000  2.84030
2020-01-04  0.00000  0.29204  0.29204
2020-01-05 -5.20050 -5.20050  0.00000
                  a        b        c
2020-01-01  0.00000 -1.12010 -1.12010
2020-01-02 -0.30402 -0.30402 -0.30402
2020-01-03  0.19403  0.00000  2.84030
2020-01-04  0.00000  0.29204  0.29204
2020-01-05 -5.20050 -5.20050  0.00000
               first   second
2020-01-01  -1.12010 -1.12010
2020-01-02  -0.60804 -0.30402
2020-01-03   0.19403  2.84030
2020-01-04   0.29204  0.29204
2020-01-05 -10.40100  0.00000
               first   second
2020-01-01  -1.12010 -1.12010
2020-01-02  -0.60804 -0.30402
2020-01-03   0.19403  2.84030
2020-01-04   0.2

In [121]:
%timeit big_portfolio.cash_flow()
%timeit big_portfolio_grouped.cash_flow(group_by=False)
%timeit big_portfolio_shared.cash_flow(group_by=False)

%timeit big_portfolio.cash_flow(group_by=big_group_by)
%timeit big_portfolio_grouped.cash_flow()
%timeit big_portfolio_shared.cash_flow()

8.36 ms ± 68.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.4 ms ± 76.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.66 ms ± 200 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
14.2 ms ± 134 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
14.4 ms ± 192 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
15.5 ms ± 2.2 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [122]:
print(portfolio.cash())
print(portfolio_grouped.cash(group_by=False))
print(portfolio_shared.cash(group_by=False))
print(portfolio_shared.cash(group_by=False, in_sim_order=True))

print(portfolio.cash(group_by=group_by))
print(portfolio_grouped.cash())
print(portfolio_shared.cash())

                    a         b          c
2020-01-01  100.00000  98.87990   98.87990
2020-01-02   99.69598  98.57588   98.57588
2020-01-03   99.89001  98.57588  101.41618
2020-01-04   99.89001  98.86792  101.70822
2020-01-05   94.68951  93.66742  101.70822
                    a         b          c
2020-01-01  100.00000  98.87990   98.87990
2020-01-02   99.69598  98.57588   98.57588
2020-01-03   99.89001  98.57588  101.41618
2020-01-04   99.89001  98.86792  101.70822
2020-01-05   94.68951  93.66742  101.70822
                    a          b          c
2020-01-01  200.00000  198.87990   98.87990
2020-01-02  199.69598  198.57588   98.57588
2020-01-03  199.89001  198.57588  101.41618
2020-01-04  199.89001  198.86792  101.70822
2020-01-05  194.68951  193.66742  101.70822
                    a          b          c
2020-01-01  198.87990  198.87990   98.87990
2020-01-02  198.27186  198.57588   98.57588
2020-01-03  198.46589  198.27186  101.41618
2020-01-04  198.75793  198.75793  101.70822


In [123]:
%timeit big_portfolio.cash()
%timeit big_portfolio_grouped.cash(group_by=False)
%timeit big_portfolio_shared.cash(group_by=False)
%timeit big_portfolio_shared.cash(group_by=False, in_sim_order=True)

%timeit big_portfolio.cash(group_by=big_group_by)
%timeit big_portfolio_grouped.cash()
%timeit big_portfolio_shared.cash()

22.4 ms ± 675 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
25.1 ms ± 1.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
25.4 ms ± 950 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
22.6 ms ± 585 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
20.2 ms ± 848 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
20.3 ms ± 461 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
21.5 ms ± 1.61 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [124]:
print(portfolio.share_flow())
print(portfolio_grouped.share_flow())
print(portfolio_shared.share_flow())

              a    b    c
2020-01-01  0.0  1.0  1.0
2020-01-02  0.1  0.1  0.1
2020-01-03 -0.1  0.0 -1.0
2020-01-04  0.0 -0.1 -0.1
2020-01-05  1.0  1.0  0.0
              a    b    c
2020-01-01  0.0  1.0  1.0
2020-01-02  0.1  0.1  0.1
2020-01-03 -0.1  0.0 -1.0
2020-01-04  0.0 -0.1 -0.1
2020-01-05  1.0  1.0  0.0
              a    b    c
2020-01-01  0.0  1.0  1.0
2020-01-02  0.1  0.1  0.1
2020-01-03 -0.1  0.0 -1.0
2020-01-04  0.0 -0.1 -0.1
2020-01-05  1.0  1.0  0.0


In [125]:
%timeit big_portfolio.share_flow()
%timeit big_portfolio_grouped.share_flow()
%timeit big_portfolio_shared.share_flow()

10.2 ms ± 246 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
10.2 ms ± 112 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
10.4 ms ± 126 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [126]:
print(portfolio.shares())
print(portfolio_grouped.shares())
print(portfolio_shared.shares())

              a    b    c
2020-01-01  0.0  1.0  1.0
2020-01-02  0.1  1.1  1.1
2020-01-03  0.0  1.1  0.1
2020-01-04  0.0  1.0  0.0
2020-01-05  1.0  2.0  0.0
              a    b    c
2020-01-01  0.0  1.0  1.0
2020-01-02  0.1  1.1  1.1
2020-01-03  0.0  1.1  0.1
2020-01-04  0.0  1.0  0.0
2020-01-05  1.0  2.0  0.0
              a    b    c
2020-01-01  0.0  1.0  1.0
2020-01-02  0.1  1.1  1.1
2020-01-03  0.0  1.1  0.1
2020-01-04  0.0  1.0  0.0
2020-01-05  1.0  2.0  0.0


In [127]:
%timeit big_portfolio.shares()
%timeit big_portfolio_grouped.shares()
%timeit big_portfolio_shared.shares()

17.6 ms ± 183 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
17.8 ms ± 196 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
17.9 ms ± 117 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [128]:
portfolio.orders().count()

a    3
b    4
c    4
dtype: int64

In [131]:
print(portfolio.orders().count())
print(portfolio_grouped.orders(group_by=False).count())
print(portfolio_shared.orders(group_by=False).count())

print(portfolio.orders(group_by=group_by).count())
print(portfolio_grouped.orders().count())
print(portfolio_shared.orders().count())

a    3
b    4
c    4
dtype: int64
a    3
b    4
c    4
dtype: int64
a    3
b    4
c    4
dtype: int64
first     7
second    4
dtype: int64
first     7
second    4
dtype: int64
first     7
second    4
dtype: int64


In [132]:
print(portfolio.trades(incl_unrealized=True).count())

print(portfolio.trades().count())
print(portfolio_grouped.trades(group_by=False).count())
print(portfolio_shared.trades(group_by=False).count())

print(portfolio.trades(group_by=group_by).count())
print(portfolio_grouped.trades().count())
print(portfolio_shared.trades().count())

a    2
b    2
c    2
dtype: int64
a    1
b    1
c    2
dtype: int64
a    1
b    1
c    2
dtype: int64
a    1
b    1
c    2
dtype: int64
first     2
second    2
dtype: int64
first     2
second    2
dtype: int64
first     2
second    2
dtype: int64


In [133]:
%timeit big_portfolio.trades()

The slowest run took 6.09 times longer than the fastest. This could mean that an intermediate result is being cached.
120 ms ± 65.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [134]:
print(portfolio.positions(incl_unrealized=True).count())

print(portfolio.positions().count())
print(portfolio_grouped.positions(group_by=False).count())
print(portfolio_shared.positions(group_by=False).count())

print(portfolio.positions(group_by=group_by).count())
print(portfolio_grouped.positions().count())
print(portfolio_shared.positions().count())

a    2
b    1
c    1
dtype: int64
a    1
b    0
c    1
dtype: int64
a    1
b    0
c    1
dtype: int64
a    1
b    0
c    1
dtype: int64
first     1
second    1
dtype: int64
first     1
second    1
dtype: int64
first     1
second    1
dtype: int64


In [135]:
%timeit big_portfolio.positions()

42 ms ± 13.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [136]:
print(portfolio.drawdowns().count())
print(portfolio_grouped.drawdowns(group_by=False).count())
print(portfolio_shared.drawdowns(group_by=False).count())

print(portfolio.drawdowns(group_by=group_by).count())
print(portfolio_grouped.drawdowns().count())
print(portfolio_shared.drawdowns().count())

a    1
b    0
c    1
dtype: int64
a    1
b    0
c    1
dtype: int64
a    1
b    0
c    1
dtype: int64
first     0
second    1
dtype: int64
first     0
second    1
dtype: int64
first     0
second    1
dtype: int64


In [137]:
%timeit big_portfolio.drawdowns()

55.9 ms ± 1.03 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [138]:
print(portfolio.holding_value())
print(portfolio_grouped.holding_value(group_by=False))
print(portfolio_shared.holding_value(group_by=False))

print(portfolio.holding_value(group_by=group_by))
print(portfolio_grouped.holding_value())
print(portfolio_shared.holding_value())

              a     b    c
2020-01-01  0.0   1.0  1.0
2020-01-02  0.2   2.2  2.2
2020-01-03  0.0   NaN  0.3
2020-01-04  0.0   4.0  0.0
2020-01-05  5.0  10.0  0.0
              a     b    c
2020-01-01  0.0   1.0  1.0
2020-01-02  0.2   2.2  2.2
2020-01-03  0.0   NaN  0.3
2020-01-04  0.0   4.0  0.0
2020-01-05  5.0  10.0  0.0
              a     b    c
2020-01-01  0.0   1.0  1.0
2020-01-02  0.2   2.2  2.2
2020-01-03  0.0   NaN  0.3
2020-01-04  0.0   4.0  0.0
2020-01-05  5.0  10.0  0.0
            first  second
2020-01-01    1.0     1.0
2020-01-02    2.4     2.2
2020-01-03    NaN     0.3
2020-01-04    4.0     0.0
2020-01-05   15.0     0.0
            first  second
2020-01-01    1.0     1.0
2020-01-02    2.4     2.2
2020-01-03    NaN     0.3
2020-01-04    4.0     0.0
2020-01-05   15.0     0.0
            first  second
2020-01-01    1.0     1.0
2020-01-02    2.4     2.2
2020-01-03    NaN     0.3
2020-01-04    4.0     0.0
2020-01-05   15.0     0.0


In [139]:
%timeit big_portfolio.holding_value()
%timeit big_portfolio_grouped.holding_value(group_by=False)
%timeit big_portfolio_shared.holding_value(group_by=False)

%timeit big_portfolio.holding_value(group_by=big_group_by)
%timeit big_portfolio_grouped.holding_value()
%timeit big_portfolio_shared.holding_value()

23.7 ms ± 386 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
24.1 ms ± 313 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
24.1 ms ± 291 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
35.8 ms ± 292 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
48.6 ms ± 5.87 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
37.2 ms ± 991 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [140]:
print(portfolio.value())
print(portfolio_grouped.value(group_by=False))
print(portfolio_shared.value(group_by=False))
print(portfolio_shared.value(group_by=False, in_sim_order=True))

print(portfolio.value(group_by=group_by))
print(portfolio_grouped.value())
print(portfolio_shared.value())

                    a          b          c
2020-01-01  100.00000   99.87990   99.87990
2020-01-02   99.89598  100.77588  100.77588
2020-01-03   99.89001        NaN  101.71618
2020-01-04   99.89001  102.86792  101.70822
2020-01-05   99.68951  103.66742  101.70822
                    a          b          c
2020-01-01  100.00000   99.87990   99.87990
2020-01-02   99.89598  100.77588  100.77588
2020-01-03   99.89001        NaN  101.71618
2020-01-04   99.89001  102.86792  101.70822
2020-01-05   99.68951  103.66742  101.70822
                    a          b          c
2020-01-01  200.00000  199.87990   99.87990
2020-01-02  199.89598  200.77588  100.77588
2020-01-03  199.89001        NaN  101.71618
2020-01-04  199.89001  202.86792  101.70822
2020-01-05  199.68951  203.66742  101.70822
                    a          b          c
2020-01-01  199.87990  199.87990   99.87990
2020-01-02  200.67186  200.77588  100.77588
2020-01-03        NaN        NaN  101.71618
2020-01-04  202.75793  202.75793

In [141]:
%timeit big_portfolio.value()
%timeit big_portfolio_grouped.value(group_by=False)
%timeit big_portfolio_shared.value(group_by=False)
%timeit big_portfolio_shared.value(group_by=False, in_sim_order=True)

%timeit big_portfolio.value(group_by=big_group_by)
%timeit big_portfolio_grouped.value()
%timeit big_portfolio_shared.value()

53.2 ms ± 7.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
61.7 ms ± 11.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
64.1 ms ± 7.89 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
82.1 ms ± 6.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
59.9 ms ± 1.57 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
59.9 ms ± 664 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
60.1 ms ± 469 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [142]:
print(portfolio.final_value())
print(portfolio_grouped.final_value(group_by=False))
print(portfolio_shared.final_value(group_by=False))

print(portfolio.final_value(group_by=group_by))
print(portfolio_grouped.final_value())
print(portfolio_shared.final_value())

a     99.68951
b    103.66742
c    101.70822
dtype: float64
a     99.68951
b    103.66742
c    101.70822
dtype: float64
a    199.68951
b    203.66742
c    101.70822
dtype: float64
first     203.35693
second    101.70822
dtype: float64
first     203.35693
second    101.70822
dtype: float64
first     203.35693
second    101.70822
dtype: float64


In [143]:
%timeit big_portfolio.final_value()
%timeit big_portfolio_grouped.final_value(group_by=False)
%timeit big_portfolio_shared.final_value(group_by=False)

%timeit big_portfolio.final_value(group_by=big_group_by)
%timeit big_portfolio_grouped.final_value()
%timeit big_portfolio_shared.final_value()

7 ms ± 889 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
7.04 ms ± 112 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.22 ms ± 847 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
7.59 ms ± 168 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.98 ms ± 136 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.17 ms ± 172 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [144]:
print(portfolio.total_profit())
print(portfolio_grouped.total_profit(group_by=False))
print(portfolio_shared.total_profit(group_by=False))

print(portfolio.total_profit(group_by=group_by))
print(portfolio_grouped.total_profit())
print(portfolio_shared.total_profit())

a   -0.31049
b    3.66742
c    1.70822
dtype: float64
a   -0.31049
b    3.66742
c    1.70822
dtype: float64
a   -0.31049
b    3.66742
c    1.70822
dtype: float64
first     3.35693
second    1.70822
dtype: float64
first     3.35693
second    1.70822
dtype: float64
first     3.35693
second    1.70822
dtype: float64


In [145]:
%timeit big_portfolio.total_profit()
%timeit big_portfolio_grouped.total_profit(group_by=False)
%timeit big_portfolio_shared.total_profit(group_by=False)

%timeit big_portfolio.total_profit(group_by=big_group_by)
%timeit big_portfolio_grouped.total_profit()
%timeit big_portfolio_shared.total_profit()

6.53 ms ± 758 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.37 ms ± 1.08 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
6.72 ms ± 391 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
6.62 ms ± 195 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.54 ms ± 658 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.27 ms ± 314 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [146]:
print(portfolio.total_return())
print(portfolio_grouped.total_return(group_by=False))
print(portfolio_shared.total_return(group_by=False))

print(portfolio.total_return(group_by=group_by))
print(portfolio_grouped.total_return())
print(portfolio_shared.total_return())

a   -0.003105
b    0.036674
c    0.017082
dtype: float64
a   -0.003105
b    0.036674
c    0.017082
dtype: float64
a   -0.001552
b    0.018337
c    0.017082
dtype: float64
first     0.016785
second    0.017082
dtype: float64
first     0.016785
second    0.017082
dtype: float64
first     0.016785
second    0.017082
dtype: float64


In [147]:
%timeit big_portfolio.total_return()
%timeit big_portfolio_grouped.total_return(group_by=False)
%timeit big_portfolio_shared.total_return(group_by=False)

%timeit big_portfolio.total_return(group_by=big_group_by)
%timeit big_portfolio_grouped.total_return()
%timeit big_portfolio_shared.total_return()

6.55 ms ± 501 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
7.06 ms ± 113 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.27 ms ± 139 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.42 ms ± 144 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
9.17 ms ± 1.48 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
9.52 ms ± 1.9 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [148]:
print(portfolio.buy_and_hold_return())
print(portfolio_grouped.buy_and_hold_return(group_by=False))
print(portfolio_shared.buy_and_hold_return(group_by=False))

print(portfolio.buy_and_hold_return(group_by=group_by))
print(portfolio_grouped.buy_and_hold_return())
print(portfolio_shared.buy_and_hold_return())

a    1.5
b    4.0
c    3.0
dtype: float64
a    1.5
b    4.0
c    3.0
dtype: float64
a    1.5
b    4.0
c    3.0
dtype: float64
first     2.75
second    3.00
dtype: float64
first     2.75
second    3.00
dtype: float64
first     2.75
second    3.00
dtype: float64


In [149]:
%timeit big_portfolio.buy_and_hold_return()
%timeit big_portfolio_grouped.buy_and_hold_return(group_by=False)
%timeit big_portfolio_shared.buy_and_hold_return(group_by=False)

%timeit big_portfolio.buy_and_hold_return(group_by=big_group_by)
%timeit big_portfolio_grouped.buy_and_hold_return()
%timeit big_portfolio_shared.buy_and_hold_return()

328 µs ± 7.38 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
447 µs ± 15.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
448 µs ± 15.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
810 µs ± 16.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
818 µs ± 8.91 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
845 µs ± 11.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [150]:
print(portfolio.active_returns())
print(portfolio_grouped.active_returns(group_by=False))
print(portfolio_shared.active_returns(group_by=False))

print(portfolio.active_returns(group_by=group_by))
print(portfolio_grouped.active_returns())
print(portfolio_shared.active_returns())

                   a         b         c
2020-01-01  0.000000 -0.107223 -0.107223
2020-01-02 -0.342149  0.687091  0.687091
2020-01-03 -0.029850       NaN  0.427409
2020-01-04  0.000000       NaN -0.026533
2020-01-05 -0.038554  0.086897  0.000000
                   a         b         c
2020-01-01  0.000000 -0.107223 -0.107223
2020-01-02 -0.342149  0.687091  0.687091
2020-01-03 -0.029850       NaN  0.427409
2020-01-04  0.000000       NaN -0.026533
2020-01-05 -0.038554  0.086897  0.000000
                   a         b         c
2020-01-01  0.000000 -0.107223 -0.107223
2020-01-02 -0.342149  0.687091  0.687091
2020-01-03 -0.029850       NaN  0.427409
2020-01-04  0.000000       NaN -0.026533
2020-01-05 -0.038554  0.086897  0.000000
               first    second
2020-01-01 -0.107223 -0.107223
2020-01-02  0.492500  0.687091
2020-01-03       NaN  0.427409
2020-01-04       NaN -0.026533
2020-01-05  0.041594  0.000000
               first    second
2020-01-01 -0.107223 -0.107223
2020-01-02  0.

In [151]:
%timeit big_portfolio.active_returns()
%timeit big_portfolio_grouped.active_returns(group_by=False)
%timeit big_portfolio_shared.active_returns(group_by=False)

%timeit big_portfolio.active_returns(group_by=big_group_by)
%timeit big_portfolio_grouped.active_returns()
%timeit big_portfolio_shared.active_returns()

50.9 ms ± 3.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
58 ms ± 6.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
56.7 ms ± 6.98 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
61.9 ms ± 9.09 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
57.3 ms ± 1.83 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
56.9 ms ± 596 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [152]:
print(portfolio.returns())
print(portfolio_grouped.returns(group_by=False))
print(portfolio_shared.returns(group_by=False))
print(portfolio_shared.returns(group_by=False, in_sim_order=True))

print(portfolio.returns(group_by=group_by))
print(portfolio_grouped.returns())
print(portfolio_shared.returns())

                   a         b         c
2020-01-01  0.000000 -0.001201 -0.001201
2020-01-02 -0.001040  0.008971  0.008971
2020-01-03 -0.000060       NaN  0.009331
2020-01-04  0.000000       NaN -0.000078
2020-01-05 -0.002007  0.007772  0.000000
                   a         b         c
2020-01-01  0.000000 -0.001201 -0.001201
2020-01-02 -0.001040  0.008971  0.008971
2020-01-03 -0.000060       NaN  0.009331
2020-01-04  0.000000       NaN -0.000078
2020-01-05 -0.002007  0.007772  0.000000
                   a         b         c
2020-01-01  0.000000 -0.000601 -0.001201
2020-01-02 -0.000520  0.004483  0.008971
2020-01-03 -0.000030       NaN  0.009331
2020-01-04  0.000000       NaN -0.000078
2020-01-05 -0.001003  0.003941  0.000000
                   a         b         c
2020-01-01  0.000000 -0.000601 -0.001201
2020-01-02 -0.000518  0.004483  0.008971
2020-01-03       NaN       NaN  0.009331
2020-01-04  0.000000       NaN -0.000078
2020-01-05 -0.000985  0.003943  0.000000
               f

In [153]:
%timeit big_portfolio.returns()
%timeit big_portfolio_grouped.returns(group_by=False)
%timeit big_portfolio_shared.returns(group_by=False)
%timeit big_portfolio_shared.returns(group_by=False, in_sim_order=True)

%timeit big_portfolio.returns(group_by=big_group_by)
%timeit big_portfolio_grouped.returns()
%timeit big_portfolio_shared.returns()

58.2 ms ± 1.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
59.6 ms ± 982 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
59.3 ms ± 622 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
94.8 ms ± 889 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
62.1 ms ± 1.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
62.2 ms ± 1.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
63.7 ms ± 1.71 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [154]:
print(portfolio_shared.cumulative_returns())
print(portfolio_shared.cumulative_returns(group_by=False))
print(portfolio_shared.cumulative_returns(group_by=False, active_returns=True))

               first    second
2020-01-01 -0.000601 -0.001201
2020-01-02  0.003359  0.007759
2020-01-03  0.003359  0.017162
2020-01-04  0.003359  0.017082
2020-01-05  0.006323  0.017082
                   a         b         c
2020-01-01  0.000000 -0.000601 -0.001201
2020-01-02 -0.000520  0.003879  0.007759
2020-01-03 -0.000550  0.003879  0.017162
2020-01-04 -0.000550  0.003879  0.017082
2020-01-05 -0.001552  0.007836  0.017082
                   a         b         c
2020-01-01  0.000000 -0.107223 -0.107223
2020-01-02 -0.342149  0.506196  0.506196
2020-01-03 -0.361785  0.506196  1.149959
2020-01-04 -0.361785  0.506196  1.092913
2020-01-05 -0.386391  0.637081  1.092913


In [155]:
print(portfolio_shared.sharpe_ratio())
print(portfolio_shared.sharpe_ratio(risk_free=0.01))
print(portfolio_shared.sharpe_ratio(year_freq='252D'))
print(portfolio_shared.sharpe_ratio(group_by=False))
print(portfolio_shared.sharpe_ratio(group_by=False, active_returns=True))

first     16.780949
second    12.345065
dtype: float64
first    -62.926646
second   -23.917188
dtype: float64
first     13.943454
second    10.257635
dtype: float64
a   -13.309506
b    17.846729
c    12.345065
dtype: float64
a   -10.715260
b    10.254119
c    10.879217
dtype: float64


In [156]:
print(portfolio.stats())  # mean statistics

Start                     2020-01-01 00:00:00
End                       2020-01-05 00:00:00
Duration                      5 days 00:00:00
Holding Duration [%]                  66.6667
Total Profit                          1.68838
Total Return [%]                      1.68838
Buy & Hold Return [%]                 283.333
Max. Drawdown [%]                    0.106105
Avg. Drawdown [%]                    0.106105
Max. Drawdown Duration        3 days 00:00:00
Avg. Drawdown Duration        3 days 00:00:00
Num. Trades                           1.33333
Win Rate [%]                          66.6667
Best Trade [%]                        71.6563
Worst Trade [%]                        69.594
Avg. Trade [%]                        70.6251
Max. Trade Duration           2 days 08:00:00
Avg. Trade Duration           2 days 04:00:00
Expectancy                           0.302232
SQN                                   1.23509
Sharpe Ratio                          5.61381
Sortino Ratio                     

In [157]:
print(portfolio['a'].stats())

Start                     2020-01-01 00:00:00
End                       2020-01-05 00:00:00
Duration                      5 days 00:00:00
Holding Duration [%]                       40
Total Profit                         -0.31049
Total Return [%]                     -0.31049
Buy & Hold Return [%]                     150
Max. Drawdown [%]                     0.31049
Avg. Drawdown [%]                     0.31049
Max. Drawdown Duration        4 days 00:00:00
Avg. Drawdown Duration        4 days 00:00:00
Num. Trades                                 1
Win Rate [%]                                0
Best Trade [%]                       -36.1785
Worst Trade [%]                      -36.1785
Avg. Trade [%]                       -36.1785
Max. Trade Duration           1 days 00:00:00
Avg. Trade Duration           1 days 00:00:00
Expectancy                           -0.10999
SQN                                       NaN
Sharpe Ratio                          -13.308
Sortino Ratio                     

In [158]:
print(portfolio['a'].stats(required_return=0.1, risk_free=0.01))  # test kwargs

Start                     2020-01-01 00:00:00
End                       2020-01-05 00:00:00
Duration                      5 days 00:00:00
Holding Duration [%]                       40
Total Profit                         -0.31049
Total Return [%]                     -0.31049
Buy & Hold Return [%]                     150
Max. Drawdown [%]                     0.31049
Avg. Drawdown [%]                     0.31049
Max. Drawdown Duration        4 days 00:00:00
Avg. Drawdown Duration        4 days 00:00:00
Num. Trades                                 1
Win Rate [%]                                0
Best Trade [%]                       -36.1785
Worst Trade [%]                      -36.1785
Avg. Trade [%]                       -36.1785
Max. Trade Duration           1 days 00:00:00
Avg. Trade Duration           1 days 00:00:00
Expectancy                           -0.10999
SQN                                       NaN
Sharpe Ratio                         -227.459
Sortino Ratio                     

In [159]:
print(portfolio['a'].stats(active_returns=True))  # test active_returns

Start                     2020-01-01 00:00:00
End                       2020-01-05 00:00:00
Duration                      5 days 00:00:00
Holding Duration [%]                       40
Total Profit                         -0.31049
Total Return [%]                     -0.31049
Buy & Hold Return [%]                     150
Max. Drawdown [%]                     0.31049
Avg. Drawdown [%]                     0.31049
Max. Drawdown Duration        4 days 00:00:00
Avg. Drawdown Duration        4 days 00:00:00
Num. Trades                                 1
Win Rate [%]                                0
Best Trade [%]                       -36.1785
Worst Trade [%]                      -36.1785
Avg. Trade [%]                       -36.1785
Max. Trade Duration           1 days 00:00:00
Avg. Trade Duration           1 days 00:00:00
Expectancy                           -0.10999
SQN                                       NaN
Sharpe Ratio                         -10.7153
Sortino Ratio                     

In [160]:
print(portfolio['a'].stats(incl_unrealized=True))  # test incl_unrealized

Start                     2020-01-01 00:00:00
End                       2020-01-05 00:00:00
Duration                      5 days 00:00:00
Holding Duration [%]                       40
Total Profit                         -0.31049
Total Return [%]                     -0.31049
Buy & Hold Return [%]                     150
Max. Drawdown [%]                     0.31049
Avg. Drawdown [%]                     0.31049
Max. Drawdown Duration        4 days 00:00:00
Avg. Drawdown Duration        4 days 00:00:00
Num. Trades                                 2
Win Rate [%]                                0
Best Trade [%]                        -3.8554
Worst Trade [%]                      -36.1785
Avg. Trade [%]                        -20.017
Max. Trade Duration           1 days 00:00:00
Avg. Trade Duration           0 days 12:00:00
Expectancy                          -0.155245
SQN                                  -3.43045
Sharpe Ratio                          -13.308
Sortino Ratio                     

In [161]:
pd.testing.assert_series_equal(portfolio['c'].stats(), portfolio.stats(column='c'))
pd.testing.assert_series_equal(portfolio['c'].stats(), portfolio_grouped.stats(column='c', group_by=False))
pd.testing.assert_series_equal(portfolio_grouped['second'].stats(), portfolio_grouped.stats(column='second'))

In [162]:
# Calculates stats for one column
%timeit big_portfolio.iloc[0].stats()
%timeit big_portfolio_grouped.iloc[0].stats(group_by=False, column=0)
%timeit big_portfolio_shared.iloc[0].stats(group_by=False, column=0)

%timeit big_portfolio.iloc[0].stats(group_by=np.array([0]))
%timeit big_portfolio_grouped.iloc[0].stats()
%timeit big_portfolio_shared.iloc[0].stats()

19.2 ms ± 199 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
22.1 ms ± 324 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
23.5 ms ± 724 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
32.1 ms ± 1.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
29.3 ms ± 340 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
30.6 ms ± 611 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [163]:
# Calculates stats for all columns and selects one, takes advantage of caching (which is disabled here)
# Thus some series are re-calculated multiple times, such as records and returns
%timeit big_portfolio.stats(column=0)
%timeit big_portfolio_grouped.stats(group_by=False, column=0)
%timeit big_portfolio_shared.stats(group_by=False, column=0)

%timeit big_portfolio.stats(group_by=big_group_by, column=0)
%timeit big_portfolio_grouped.stats(column=0)
%timeit big_portfolio_shared.stats(column=0)

351 ms ± 2.08 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
367 ms ± 7.06 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
370 ms ± 12.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
364 ms ± 2.96 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
367 ms ± 6.28 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
368 ms ± 4.15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
